<a href="https://colab.research.google.com/github/thomasmcz/SEBAL_STARFM/blob/main/GitHub_SEBAL_Script_Pet_Juaz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Author contact information

In [ ]:
'''
Thomás R Ferreira, Dr. in Meteorology
UACA/CTRN/UFCG
thomasmcz@gmail.com
thomas.ferreira@ufcg.edu.br                                        
Last updated on June 03, 2022
'''

# INITIALIZING EE

In [ ]:
# Import Earth Engine Library
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Importing libraries and defining functions

In [ ]:
# Import some essencial libraries and define the functions for focal analyses (make_slices) and image export (make_raster)

import numpy as np
from osgeo import gdal
import os
import math

os.chdir(r"/content/drive/MyDrive/colab_imagens/Pet_Juaz")

# The next function was obtained from "Geoprocessing with Python" book, by Chris Garrard
# It is available from https://www.manning.com/books/geoprocessing-with-python?query=geoprocessing
 
def make_raster(in_ds, fn, data, data_type, nodata=None):
    """Create a one-band GeoTIFF.
    in_ds - datasource to copy projection and geotransform from
    fn - path to the file to create
    data - NumPy array containing data to write
    data_type - output data type
    nodata - optional NoData value"""
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(fn, in_ds.RasterXSize, in_ds.RasterYSize, 1, data_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    out_band = out_ds.GetRasterBand(1)
    if nodata is not None:
        out_band.SetNoDataValue(nodata)
    out_band.WriteArray(data)
    out_band.FlushCache()
    out_band.ComputeStatistics(False)
    return out_ds

# Exporting Landsat-8 images to your drive

In [ ]:
# Define your study area
os.chdir(r"/content/drive/MyDrive/colab_imagens")

def maskL8sr(image):
  cloudShadowBitMask = ee.Number(1 << 3)#converting 1 to binary (0001) and adding the 3 zeros after the binary number (0001000), which means 2^3
  cloudsBitMask = ee.Number(1 << 5)#converting 1 to binary (0001) and adding the 5 zeros after the binary number (000100000), which means 2^5
  qa = image.select(ee.String('pixel_qa'))
  mask0 = qa.bitwiseAnd(cloudShadowBitMask).eq(0)
  mask = qa.bitwiseAnd(cloudsBitMask).eq(0)
  image1 = image.updateMask(mask0)
  return image1.updateMask(mask)

area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])
#Choose the dates of the Landsat images that will be processed in the STARFM algorithm

lista_de_datas = ['2015-08-08', '2015-08-24', '2015-09-09', '2015-09-25', '2015-10-11', '2015-10-27', '2015-11-12', '2015-12-14', '2016-01-31', '2016-02-16', '2016-03-03', '2016-05-06', '2016-05-22']

numero_de_datas = len(lista_de_datas)

for dat in range(0, numero_de_datas):
  x = lista_de_datas[dat]
  year = int(x[:4])
  month = int(x[5:-3])
  day = int(x[-2:])

  if year == 1984 or year == 1988 or year == 1992 or year == 1996 or year == 2000 or year == 2004 or year == 2008 or year == 2012 or year == 2016 or year == 2020:
    if month == 2:
      if day == 29:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  else:
    if month == 2:
      if day == 28:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)

  img = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(lista_de_datas[dat], data2).filterBounds(area).map(maskL8sr)
  raw = ee.ImageCollection('LANDSAT/LC08/C01/T1').filterDate(lista_de_datas[dat], data2).filterBounds(area)
  mosaico = img.mosaic()
  surface_reflectance = mosaico.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']).multiply(0.0001)
  albedo = surface_reflectance.expression(
    '(0.2453*B2) + (0.0508*B3) + (0.1804*B4) + (0.3081*B5) + (0.1332*B6) + (0.0521*B7) + 0.0011',{'B2': surface_reflectance.select('B2'),'B3': surface_reflectance.select('B3'),'B4': surface_reflectance.select('B4'),'B5': surface_reflectance.select('B5'),'B6': surface_reflectance.select('B6'),'B7': surface_reflectance.select('B7')})

  tb =  mosaico.select(['B10']).multiply(0.1)
  rad_10 = tb.expression('(774.89/(exp(1321.08/tb)-1)) - 0.25', {'tb': tb.select('B10')})

  add = ee.Number(raw.aggregate_mean('RADIANCE_ADD_BAND_10'))
  mult = ee.Number(raw.aggregate_mean('RADIANCE_MULT_BAND_10'))

  mosaico_raw = raw.mosaic()

  rad = mosaico_raw.expression('add + mult * nd',{'add': add, 'mult': mult, 'nd': mosaico_raw.select('B10')})

  elevacao = raw.aggregate_mean('SUN_ELEVATION')
  cos_z = ee.Number(elevacao).multiply(math.pi).divide(180).sin()
  dt_s = raw.aggregate_mean('EARTH_SUN_DISTANCE')
  dr = ee.Number(1).divide((ee.Number(dt_s)).pow(2))

  reflectance_toa = mosaico_raw.expression('(-0.10000000149011612 + 0.000019999999494757503 * nd) / (cos_z * dr)',{'nd': mosaico_raw,'cos_z': cos_z, 'dr': dr})

  savi = reflectance_toa.expression('(1.1 * (NIR - RED)) / (0.1 + NIR + RED)',{'NIR': reflectance_toa.select('B5'), 'RED': reflectance_toa.select('B4')})

  iaf = savi.where(savi.lt(0.688000), savi.expression('(log((0.69 - savi) / 0.59)) * (-1) / 0.91',{'savi': savi.select('constant'),}))
  iaf2 = iaf.where(savi.gt(0.688000), 6)
  iafcorrigido = iaf2.where(savi.lt(0.000001), 0)

  e_nb = iafcorrigido.expression('0.97 + (0.0033 * iaf)', {'iaf': iafcorrigido.select('constant')})
  e_nb2 = e_nb.where(iafcorrigido.gt(2.999999), 0.98)
  e_nbcorrigido = e_nb2.where(iafcorrigido.lt(0.000001), 0.99)

  tsup00 = rad.expression('1321.08 / log(774.89 * e_nb / (radiance - 0.29) + 1)', {'e_nb': e_nbcorrigido.select('constant'), 'radiance': rad})
  tsup = tsup00.updateMask(surface_reflectance.select('B2').neq(0))

  imagem_1 = ee.String('reflectance_')
  imagem_2 = ee.String('albedo_')
  imagem_3 = ee.String('tsup_filtro_nuvens_')
  imagem_4 = ee.String('rad_b10_')

  saida_1 = imagem_1.cat(lista_de_datas[dat])
  saida_3 = imagem_2.cat(lista_de_datas[dat])
  saida_5 = imagem_3.cat(lista_de_datas[dat])
  saida_7 = imagem_4.cat(lista_de_datas[dat])

  #task1 = ee.batch.Export.image.toDrive(image=surface_reflectance, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  #task3 = ee.batch.Export.image.toDrive(image=albedo, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_3.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  task5 = ee.batch.Export.image.toDrive(image=tsup, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_5.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

  #task7 = ee.batch.Export.image.toDrive(image=rad_10, folder='Pet_Juaz', description='Landsat', scale=100, region=area, fileNamePrefix=saida_7.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

  #task1.start()
  #task3.start()
  task5.start()
  #task7.start()

  print('End of the Script for', lista_de_datas[dat])

# Exporting Landsat-7 images to your drive

In [ ]:
#Surface Albedo from Shuai et al., 2014 (https://doi.org/10.1016/j.rse.2014.07.009)

# Define your study area
os.chdir(r"/content/drive/MyDrive/colab_imagens")

def cloudMaskL457(image):
  cloud1 = ee.Number(1 << 5)#converting 1 to binary (0001) and adding the 3 zeros after the binary number (0001000), which means 2^3
  cloud2 = ee.Number(1 << 7)#converting 1 to binary (0001) and adding the 5 zeros after the binary number (000100000), which means 2^5
  cloud3 = ee.Number(1 << 3)
  qa = image.select(ee.String('pixel_qa'))
  mask0 = qa.bitwiseAnd(cloud1).eq(0) and qa.bitwiseAnd(cloud2).eq(0)
  mask = qa.bitwiseAnd(cloud3).eq(0)
  image1 = image.updateMask(mask0)
  return image1.updateMask(mask)

area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

#Choose the dates of the Landsat images that will be processed in the STARFM algorithm
lista_de_datas = ['2015-07-31', '2015-09-01', '2015-10-03', '2015-11-20', '2015-12-06', '2015-12-22', '2016-02-08', '2016-02-24', '2016-03-11', '2016-04-28', '2016-05-14']

numero_de_datas = len(lista_de_datas)

for dat in range(0, numero_de_datas):
  x = lista_de_datas[dat]
  year = int(x[:4])
  month = int(x[5:-3])
  day = int(x[-2:])

  if year == 1984 or year == 1988 or year == 1992 or year == 1996 or year == 2000 or year == 2004 or year == 2008 or year == 2012 or year == 2016 or year == 2020:
    if month == 2:
      if day == 29:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  else:
    if month == 2:
      if day == 28:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)

  img = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate(lista_de_datas[dat], data2).filterBounds(area).map(cloudMaskL457)
  mosaico = img.mosaic()

  raw = ee.ImageCollection('LANDSAT/LE07/C01/T1').filterDate(lista_de_datas[dat], data2).filterBounds(area)

  #Surface Albedo from Shuai et al., 2014 (https://doi.org/10.1016/j.rse.2014.07.009)
  surface_reflectance = mosaico.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7']).multiply(0.0001)
  albedo = surface_reflectance.expression(
    '(0.3141*B1) + (0.1607*B3) + (0.3694*B4) + (0.1160*B5) + (0.0456*B7) - 0.0057',{'B1': surface_reflectance.select('B1'),'B3': surface_reflectance.select('B3'),'B4': surface_reflectance.select('B4'),'B5': surface_reflectance.select('B5'),'B7': surface_reflectance.select('B7'),})

  tb =  mosaico.select(['B6']).multiply(0.1)
  rad_10 = tb.expression('(666.09/(exp(1282.71/tb)-1))', {'tb': tb.select('B6')})

  add = ee.Number(raw.aggregate_mean('RADIANCE_ADD_BAND_6_VCID_2'))
  mult = ee.Number(raw.aggregate_mean('RADIANCE_MULT_BAND_6_VCID_2'))

  mosaico_raw = raw.mosaic()

  rad = mosaico_raw.expression('add + mult * nd',{'add': add, 'mult': mult, 'nd': mosaico_raw.select('B6_VCID_2')})

  elevacao = raw.aggregate_mean('SUN_ELEVATION')
  cos_z = ee.Number(elevacao).multiply(math.pi).divide(180).sin()
  dt_s = raw.aggregate_mean('EARTH_SUN_DISTANCE')
  dr = ee.Number(1).divide((ee.Number(dt_s)).pow(2))

  reflectance_toa = mosaico_raw.expression('(-0.10000000149011612 + 0.000019999999494757503 * nd) / (cos_z * dr)',{'nd': mosaico_raw,'cos_z': cos_z, 'dr': dr})

  savi = reflectance_toa.expression('(1.1 * (NIR - RED)) / (0.1 + NIR + RED)',{'NIR': reflectance_toa.select('B4'), 'RED': reflectance_toa.select('B3'),})

  iaf = savi.where(savi.lt(0.688000), savi.expression('(log((0.69 - savi) / 0.59)) * (-1) / 0.91',{'savi': savi.select('constant'),}))
  iaf2 = iaf.where(savi.gt(0.688000), 6)
  iafcorrigido = iaf2.where(savi.lt(0.000001), 0)

  e_nb = iafcorrigido.expression('0.97 + (0.0033 * iaf)', {'iaf': iafcorrigido.select('constant')})
  e_nb2 = e_nb.where(iafcorrigido.gt(2.999999), 0.98)
  e_nbcorrigido = e_nb2.where(iafcorrigido.lt(0.000001), 0.99)

  tsup00 = rad.expression('1282.71 / log(666.09 * e_nb / (radiance) + 1)', {'e_nb': e_nbcorrigido.select('constant'), 'radiance': rad})
  tsup = tsup00.updateMask(surface_reflectance.select('B2').neq(0))



  imagem_1 = ee.String('reflectance_2_')
  imagem_2 = ee.String('albedo_2_')
  imagem_3 = ee.String('tsup_2_')
  imagem_4 = ee.String('rad_b10_')

  saida_1 = imagem_1.cat(lista_de_datas[dat])
  saida_3 = imagem_2.cat(lista_de_datas[dat])
  saida_5 = imagem_3.cat(lista_de_datas[dat])
  saida_7 = imagem_4.cat(lista_de_datas[dat])

  task1 = ee.batch.Export.image.toDrive(image=surface_reflectance, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  task3 = ee.batch.Export.image.toDrive(image=albedo, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_3.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  task5 = ee.batch.Export.image.toDrive(image=tsup, folder='Pet_Juaz', description='Landsat', scale=30, region=area, fileNamePrefix=saida_5.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

  #task7 = ee.batch.Export.image.toDrive(image=rad_10, folder='Pet_Juaz', description='Landsat', scale=60, region=area, fileNamePrefix=saida_7.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

  task1.start()
  task3.start()
  task5.start()
  #task7.start()

  print('End of the Script for', lista_de_datas[dat])

# Exporting land use and land cover image

In [ ]:
os.chdir(r"/content/drive/MyDrive/colab_imagens")
data1 = ee.String('2015_2')
imagem_1 = ee.String('mapbiomas_')
saida_1 = imagem_1.cat(data1)

area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

mapbiomas = ee.Image('projects/mapbiomas-workspace/public/collection5/mapbiomas_collection50_integration_v1').select(['classification_2015'])

task1 = ee.batch.Export.image.toDrive(image=mapbiomas, folder='Pet_Juaz', description='Mapbiomas', scale=30, region=area, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
task1.start()
task1.status()

# SEBAL SCRIPT - LANDSAT 8

In [ ]:
os.chdir(r"/content/drive/MyDrive/colab_imagens/Pet_Juaz")

area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

lat = -9.457#///////// Insert the average Latitude of the scene!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

lista_de_datas = ['2015-08-08', '2015-08-24', '2015-09-09', '2015-09-25', '2015-10-11', '2015-10-27', '2015-11-12', '2015-12-14', '2016-01-31', '2016-02-16', '2016-03-03', '2016-05-06', '2016-05-22']

lista_de_DOA = [220, 236, 252, 268, 284, 300, 316, 348, 31, 47, 63, 127, 143]
lista_de_ta = [24.30, 26.60, 28.10, 29.40, 27.67, 28.60, 29.60, 29.90, 25.44, 27.64, 28.98, 29.21, 28.48]
lista_de_ur = [48.00, 47.80, 48.30, 45.65, 44.34, 45.60, 41.00, 42.00, 69.89, 73.30, 58.00, 45.35, 51.04]
lista_de_pa = [97.59, 97.30, 97.30, 97.36, 97.33, 97.30, 97.10, 97.10, 97.30, 97.11, 97.13, 97.36, 97.46]
lista_de_u = [5.40, 4.83, 3.77, 3.27, 4.27, 4.14, 5.16, 6.21, 4.21, 3.02, 4.13, 3.80, 3.90]

lista_de_rs_24 = [23.00, 25.80, 26.24, 28.64, 30.55, 30.92, 30.83, 31.31, 27.32, 19.45, 24.92, 22.63, 22.66]

lista_de_precipitation60 = [5.842, 6.096, 5.334, 0.762, 10.672, 10.418, 10.418, 2.286, 303.774, 323.84, 321.044, 13.97, 4.318]
lista_de_ETr60 = [341.3, 262.82, 289.06, 356.5, 378.7, 300.3, 292.33, 309.32, 328.3, 277.3, 250.5, 297.6, 302.01]

numero_de_datas = len(lista_de_datas)

for dat in range(0, numero_de_datas):
  x = lista_de_datas[dat]
  year = int(x[:4])
  month = int(x[5:-3])
  day = int(x[-2:])

  if year == 1984 or year == 1988 or year == 1992 or year == 1996 or year == 2000 or year == 2004 or year == 2008 or year == 2012 or year == 2016 or year == 2020:
    if month == 2:
      if day == 29:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  else:
    if month == 2:
      if day == 28:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  
  raw = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(lista_de_datas[dat], data2).filterBounds(area)
  ang_zenital = raw.aggregate_mean('SOLAR_ZENITH_ANGLE').getInfo()
  elevacao = 90 - ang_zenital
  cos_z = np.cos(ang_zenital*np.pi/180)
  dr = 1/((raw.aggregate_mean('EARTH_SUN_DISTANCE').getInfo())**2)

  print('This is the z angle: ', ang_zenital, 'This is the elevation: ', elevacao, 'This is the cos(Z): ', cos_z, 'This is the dr: ', dr)


  albedo_open = gdal.Open('albedo_' + lista_de_datas[dat] + '.tif')
  albedo_in_band = albedo_open.GetRasterBand(1)
  albedo_in_data = albedo_in_band.ReadAsArray().astype(float)

  reflectance_open = gdal.Open('reflectance_' + lista_de_datas[dat] + '.tif')
  NIR_in_band = reflectance_open.GetRasterBand(4)
  NIR_in_data = NIR_in_band.ReadAsArray().astype(float)

  RED_in_band = reflectance_open.GetRasterBand(3)
  RED_in_data = RED_in_band.ReadAsArray().astype(float)

  tsup_open = gdal.Open('tsup_' + lista_de_datas[dat] + '.tif')
  tsup_in_band = tsup_open.GetRasterBand(1)
  tsup_in_data = tsup_in_band.ReadAsArray().astype(float)

  nlcd_open = gdal.Open('mapbiomas_2015_2.tif')
  nlcd_in_band = nlcd_open.GetRasterBand(1)
  nlcd_in_data = nlcd_in_band.ReadAsArray().astype(float)

  albedo = np.ma.masked_where(albedo_in_data < 0, albedo_in_data)
  albedo = np.ma.masked_where(albedo > 1, albedo)
 
  NIR = np.where(NIR_in_data < 0.0, 0.0, NIR_in_data)
  NIR = np.ma.masked_where(NIR <= 0, NIR)
 
  RED = np.where(RED_in_data < 0.0, 0.0, RED_in_data)
  RED = np.ma.masked_where(RED <= 0, RED)
 
  tsup = np.ma.masked_where(tsup_in_data < 253, tsup_in_data)
  tsup = np.ma.masked_where(tsup > 340, tsup)
 
 
#EXPORTS#####################################################

  nome_ndvi = '_ndvi_landsat.tif'
  export_ndvi = lista_de_datas[dat] + nome_ndvi
 
  nome_et24 = '_et_24_dms_landsat_.tif'
  export_et24 = lista_de_datas[dat] + nome_et24

  nome_le = '_le_dms_landsat.tif'
  export_le = lista_de_datas[dat] + nome_le

  nome_g = '_g_dms_landsat.tif'
  export_g = lista_de_datas[dat] + nome_g

  nome_rn = '_rn_dms_landsat.tif'
  export_rn = lista_de_datas[dat] + nome_rn 

  nome_h = '_h_dms_landsat-2.tif'
  export_h = lista_de_datas[dat] + nome_h

  nome_fe = '_fe_dms_landsat.tif'
  export_fe = lista_de_datas[dat] + nome_fe

  nome_tsup_q = '_tsup_pixls_quentes.tif'
  export_tsup_q = lista_de_datas[dat] + nome_tsup_q

  nome_tsup_f = '_tsup_pixls_frios.tif'
  export_tsup_f = lista_de_datas[dat] + nome_tsup_f

  nome_tsup = '_tsup.tif'
  export_tsup = lista_de_datas[dat] + nome_tsup

  nome_albedo = '_albedo.tif'
  export_albedo = lista_de_datas[dat] + nome_albedo

  nome_rah = '_rah_corr.tif'
  export_rah = lista_de_datas[dat] + nome_rah

  nome_rn_24 = '_rn_24.tif'
  export_rn_24 = lista_de_datas[dat] + nome_rn_24

#CALCULUS####################################################
  Tfac = 2.6 - (13*(lista_de_precipitation60[dat]/lista_de_ETr60[dat]))
  print(Tfac)

  savi = (1.1 * (NIR - RED)) / (0.1 + NIR + RED)

  iaf = np.where(savi < 0.688000, np.log((0.69 - savi) / 0.59) * (-1) / 0.91, 6)
  iafcorrigido = np.where(savi < 0.000001, 0.000001, iaf)
  iafcorrigido = np.where(iafcorrigido < 0.000001, 0.000001, iaf)

  e_o = np.where(iafcorrigido > 2.999999, 0.98, 0.95 + (0.01 * iafcorrigido))
  e_ocorrigido = np.where(iafcorrigido < 0.000001, 0.985, e_o)

  e_nb = np.where(iafcorrigido > 2.999999, 0.98, 0.97 + (0.0033 * iafcorrigido))
  e_nbcorrigido = np.where(iafcorrigido < 0.000001, 0.99, e_nb)

  rol_emi = e_o * 0.0000000567 * tsup**4

  e_a = 0.61078 * (np.exp((17.269 * lista_de_ta[dat])/(237.3 + lista_de_ta[dat]))) * lista_de_ur[dat]/100
  w = (10 * (0.14 * 10 * e_a * (lista_de_pa[dat]/101))) + 0.21
  transmitancia = 0.35 + 0.627 * (np.exp((0.00146*(-1)*lista_de_pa[dat]/cos_z) - 0.075 * ((w/cos_z)**0.4)))

  print ('e_a = : ', e_a)
  print ('w = : ', w)
  print ('transmitancia = : ', transmitancia)

  roc = 1367 * dr * cos_z * transmitancia
  print ('This is the shortwave radiation: ', roc)

  e_atm = 0.625 * (((1000 * e_a) / (273.15 + lista_de_ta[dat])) ** 0.131)

  rol_atm = e_atm * 0.0000000567 * (lista_de_ta[dat] + 273.15)**4
  print ('The Rad_atm is: ', rol_atm)

  rn = (1 - albedo) * roc - rol_emi + e_ocorrigido * rol_atm 

  ndvi = (NIR - RED) / (NIR + RED)

  g_corrigido = np.where(ndvi < 0.05, 0.3 * rn, ((tsup - 273.15) * (0.0038 + 0.0074 * albedo) * (1 - 0.98 * (ndvi ** 4))) * rn)

#//#//#//#//#//#//#//#//#//#// ENERGY BALANCE#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//COLD PIXEL - CIMEC

  p95 = np.percentile(ndvi[ndvi > 0], 95)
  print('This is the 95th percentile of the NDVI: ', p95)

  f1_frio_0 = np.ma.masked_where(nlcd_in_data != 36, ndvi)
  f1_frio = np.ma.masked_where(f1_frio_0 < p95, tsup)
  
  fff = f1_frio[f1_frio > 274]

  p20 =  np.percentile(fff[fff < 350], 20)
  print('This is the 20th percentile of Tsup: ', p20)

  f2_frio = np.ma.masked_where(f1_frio > p20, f1_frio)

  f2_frio_med = np.nanmean(f2_frio[f2_frio > 0])
  print('This is the average for the cool pixel: ', f2_frio_med)

  f3_frio = np.ma.masked_where(f2_frio > (f2_frio_med + 0.3), f2_frio)
  f3_frio = np.ma.masked_where(f2_frio < (f2_frio_med - 0.3), f3_frio)

  albedo_chave = 0.001343 * elevacao + (0.3281 * np.exp((-0.0188) * elevacao))
  print('The key-albedo is: ', albedo_chave)

  f4_frio = np.ma.masked_where(albedo > (albedo_chave + 0.02), f3_frio)
  f4_frio = np.ma.masked_where(albedo < (albedo_chave - 0.02), f4_frio)

  p_frio_ok = np.min(f4_frio[f4_frio > 0])

  print('This is the CIMEC cold pixel: ', p_frio_ok)

#//HOT PIXEL - CIMEC
  p5 = np.percentile(ndvi[ndvi>0], 5)

  f1_quente = np.ma.masked_where(ndvi > p5, tsup)


  f1_quente = np.ma.masked_where(f1_quente > 400, f1_quente)

  qqq = f1_quente[f1_quente > 274]

  p80 = np.percentile(qqq[qqq < 350], 80)
  print('This is the 80th percentile of the Tsup: ', p80)
  f2_quente = np.ma.masked_where(f1_quente < p80, f1_quente)

  f2_quente_med = np.nanmean(f2_quente)

  f2_quente_med_2 = f2_quente_med - Tfac

  f3_quente = np.ma.masked_where(f2_quente > (f2_quente_med_2 + 2.0), f2_quente)
  f3_quente = np.ma.masked_where(f3_quente < (f2_quente_med_2 - 2.0), f3_quente)

  p_quente_ok = np.max(f3_quente[f3_quente > 0])


  print('This is CIMEC hot pixel: ', p_quente_ok)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///

#//SAVI OF THE HOT PIXEL
  savi_pq = np.ma.masked_where(ndvi > p5, tsup)
  savi_pq = np.ma.masked_where(savi_pq != p_quente_ok, savi)
  savi_quente = savi_pq.max()
  print ('The SAVI of the hot pixel is: ', savi_quente)

#//Rn OF THE HOT PIXEL
  rn_pq = np.ma.masked_where(ndvi > p5, tsup)
  rn_pq = np.ma.masked_where(rn_pq != p_quente_ok, rn)
  rn_quente = rn_pq.max()
  print ('The Rn of the hot pixel is: ', rn_quente)

#//G OF THE HOT PIXEL
  g_pq = np.ma.masked_where(ndvi > p5, tsup)
  g_pq = np.ma.masked_where(g_pq != p_quente_ok, g_corrigido)
  g_quente = g_pq.max()
  print ('The G of the hot pixel is: ', g_quente)

#//calculating linear regression coefficients dT = a + bT

  u_estrela1 = lista_de_u[dat] * 0.41/np.log(10/0.024)# Attention in this value 10 here in the logarithm! 10 stands for wind measurement height (10 m)
  u200 = u_estrela1 * np.log(200/0.024)/0.41
  zom = np.exp(-5.809+5.62*savi_quente)
  u_estrela = (u200*0.41)/np.log(200/zom)
  rah = (np.log(2/0.1))/(0.41 * u_estrela)
  b = (rn_quente - g_quente)*rah / (1.15*1004*(p_quente_ok - p_frio_ok))
  a = (-1)*b*(p_frio_ok - 273.15)
  dT = a + b *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a + (b * (p_quente_ok - 273.15)))/rah

  print ('The sensible heat flux H is: ', H)
  rn_g = rn_quente - g_quente
  print ('Rn - G is: ', rn_g)
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculando nas imagens#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  zom_img = np.exp(-5.809+5.62*savi)
  u_estrela_img = (u200*0.41)/np.log(200/zom_img)
  rah_img = (np.log(2/0.1))/(0.41 * u_estrela_img)
  H_img = 1.15 * 1004 * (a + (b * (tsup - 273.15)))/rah_img

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//INICIO DO PROCESSO ITERATIVO#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//1ª CORREÇÃO#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
  L = (-1)*1.15*1004*(u_estrela**3)*p_quente_ok/(0.41*9.81*H)

  x_0_1m = ((1-16)*0.1/L)**0.25
  x_2m = ((1-16)*2/L)**0.25
  x_200m = ((1-16)*200/L)**0.25

  psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
  psi_h2 = 2*np.log((1+(x_2m**2))/2)
  psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi

  u_estrela_corr_1 = u200*0.41/((np.log(200/zom))-psi_m200)
  rah_corr_1 = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*u_estrela_corr_1)
  b_corr = (rn_quente - g_quente)*rah_corr_1 / (1.15*1004*(p_quente_ok - p_frio_ok))
  a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
  dT_corr = a_corr + b_corr *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/rah_corr_1

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculando 1ª CORREÇÃO nas imagens #//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  L_img = (-1)*1.15*1004*(u_estrela_img**3)*tsup/(0.41*9.81*H_img)
  x_0_1m_img = ((1-16)*0.1/L_img)**0.25
  x_2m_img = ((1-16)*2/L_img)**0.25
  x_200m_img = ((1-16)*200/L_img)**0.25

  psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
  psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
  psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                          (-1)*5*(200/L_img))
  u_estrela_corr_img_1 = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
  rah_corr_img_1 = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * u_estrela_corr_img_1)
  H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/rah_corr_img_1

  err_rel = abs(rah_corr_1 - rah)*100/rah
  print('a = : ', a_corr, 'b = : ', b_corr, 'The relative error = : ', err_rel)
  print('H is : ', H)

  lista_u_star = []
  lista_rah = []
  lista_u_star_img = []
  lista_rah_img = []

  lista_u_star.append(u_estrela_corr_1)
  lista_rah.append(rah_corr_1)
  lista_u_star_img.append(u_estrela_corr_img_1)
  lista_rah_img.append(rah_corr_img_1)


  rodadas = 1

  while err_rel > 1.5:
    rodadas += 1
    lista_u_star.append('u_estrela_corr_{}'.format(rodadas))
    lista_rah.append('rah_corr_{}'.format(rodadas))
    lista_u_star_img.append('u_estrela_corr_img_{}'.format(rodadas))    
    lista_rah_img.append('rah_corr_img_{}'.format(rodadas))
 
    L = (-1)*1.15*1004*((lista_u_star[0])**3)*p_quente_ok/(0.41*9.81*H)
 
    x_0_1m = ((1-16)*0.1/L)**0.25
    x_2m = ((1-16)*2/L)**0.25
    x_200m = ((1-16)*200/L)**0.25
 
    psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
    psi_h2 = 2*np.log((1+(x_2m**2))/2)
    psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi
 
    lista_u_star[1] = u200*0.41/((np.log(200/zom))-psi_m200)
    lista_rah[1] = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*(lista_u_star[1]))
    b_corr = (rn_quente - g_quente)*lista_rah[1] / (1.15*1004*(p_quente_ok - p_frio_ok))
    a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
 
    H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/lista_rah[1]
 
    #calculando nas imagens:
    L_img = (-1)*1.15*1004*(((lista_u_star_img[0]))**3)*tsup/(0.41*9.81*H_corr_img)
    x_0_1m_img = ((1-16)*0.1/L_img)**0.25
    x_2m_img = ((1-16)*2/L_img)**0.25
    x_200m_img = ((1-16)*200/L_img)**0.25
 
    psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
    psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
    psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                              (-1)*5*(200/L_img))
    lista_u_star_img[1] = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
    lista_rah_img[1] = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * lista_u_star_img[1])
    H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/lista_rah_img[1]
    
    err_rel = abs(lista_rah[1] - lista_rah[0])*100/lista_rah[1]
    print('Round',rodadas,' a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)
    print ('H is: ', H)
 
    #excluíndo o primeiro ítem de cada lista
    lista_u_star.pop(0)
    lista_rah.pop(0)
    lista_u_star_img.pop(0)
    lista_rah_img.pop(0)

  print('End of the process, a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)

#//Calculando o Fluxo de calor latente (LE) e a ET_24h

  LE = rn - g_corrigido - H_corr_img
  LE = np.where(LE < 0.0, 0.0, LE)
  fe = LE / (rn - g_corrigido)

#  del_ = 0.409*np.sin(2*np.pi*lista_de_DOA[dat]/(365-1.39))
  del_ = 0.006918-0.399912*np.cos(2*np.pi*(lista_de_DOA[dat]/365))+0.070257*np.sin(2*np.pi*(lista_de_DOA[dat]/365))-0.006758*np.cos(2*2*np.pi*(lista_de_DOA[dat]/365))+0.000907*np.sin(2*2*np.pi*(lista_de_DOA[dat]/365))-0.002697*np.cos(3*2*np.pi*(lista_de_DOA[dat]/365))+0.00148*np.sin(3*2*np.pi*(lista_de_DOA[dat]/365))
  fi = lat * np.pi/180
  H_hor = np.arccos(-np.tan(fi)*np.tan(del_))
  rs_24_toa = 118.08*dr*(H_hor*np.sin(fi)*np.sin(del_)+np.cos(fi)*np.cos(del_)*np.sin(H_hor))/np.pi
  print('Rs_TOA is: ', rs_24_toa)

  tsw_24 = lista_de_rs_24[dat] / rs_24_toa
  print('tsw_24 is: ', tsw_24)

  rn_24 = (1-albedo)*lista_de_rs_24[dat]*11.57407 - 110*tsw_24 #//Rs_24 transformado em W/m²

  ET_24 = 0.0353*fe*rn_24
  ET_24 = np.ma.masked_where(ET_24 < 0, ET_24)
  ET_24 = ET_24.filled(-99)

  print('#'*150)
  print('The image(s) of the date {} is(are) beeing exported'.format(lista_de_datas[dat]))
  print('#'*150)

  out_ds = make_raster(albedo_open, export_et24, ET_24, gdal.GDT_Float32, -99)

del out_ds

# SEBAL SCRIPT - LANDSAT 7

In [ ]:
os.chdir(r"/content/drive/MyDrive/colab_imagens/Pet_Juaz")

area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

lat = -9.457#///////// Insert the average Latitude of the scene!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#lista_de_datas = ['2015-07-31', '2015-09-01', '2015-10-03', '2015-11-20', '2015-12-06', '2015-12-22', '2016-02-08', '2016-02-24', '2016-03-11', '2016-04-28', '2016-05-14']
lista_de_datas = ['2015-12-06']

#######remember to change the year of the nlcd data!!!!########

'''lista_de_DOA = [212, 244, 276, 324, 340, 356, 39, 55, 70, 119, 135]
lista_de_ta = [26.62, 27.31, 27.77, 34.06, 28.70, 28.98, 26.20, 28.53, 28.77, 30.82, 28.59]
lista_de_ur = [46.10, 50.90, 49.59, 35.54, 64.04, 47.00, 65.69, 57.57, 55.74, 48.77, 55.97]
lista_de_pa = [97.51, 97.43, 97.25, 96.67, 97.12, 97.15, 97.41, 97.12, 97.10, 97.05, 97.34]
lista_de_u = [5.09, 3.04, 3.87, 3.18, 4.51, 5.02, 4.48, 4.55, 4.65, 4.99, 2.72]

lista_de_rs_24 = [22.87, 25.86, 27.27, 27.61, 23.32, 29.68, 29.39, 29.11, 26.20, 23.55, 23.51]
#lista_de_rs_24 = [22.87, 25.86, 27.27, 27.61, 25.15, 29.68, 29.39, 29.11, 26.20, 23.55, 23.51] #Em DOA 340, Rs_24 da torre
lista_de_precipitation60 = [5.842, 6.096, 0.254, 10.672, 12.704, 2.54, 304.028, 329.429, 245.356, 13.97, 13.97]
lista_de_ETr60 = [341.9, 346.7, 369.7, 420.8, 413.1, 425.9, 306.9, 259.1, 253.3, 301.3, 294.8]
'''
############

lista_de_DOA = [340]
lista_de_ta = [28.70]
lista_de_ur = [ 64.04]
lista_de_pa = [97.12]
lista_de_u = [4.51]


lista_de_rs_24 = [25.15] #Em DOA 340, Rs_24 da torre
lista_de_precipitation60 = [12.704]
lista_de_ETr60 = [413.1]

lista_de_datas_nlcd = [2001, 2004, 2006, 2008, 2011, 2013, 2016]
data_nlcd = []

numero_de_datas = len(lista_de_datas)

for dat in range(0, numero_de_datas):
  x = lista_de_datas[dat]
  year = int(x[:4])
  month = int(x[5:-3])
  day = int(x[-2:])

  if year == 1984 or year == 1988 or year == 1992 or year == 1996 or year == 2000 or year == 2004 or year == 2008 or year == 2012 or year == 2016 or year == 2020:
    if month == 2:
      if day == 29:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  else:
    if month == 2:
      if day == 28:
        data2 = x[:5] + '03-01'
      else:
        data2 = x[:5] + '02-' + str(day + 1)
    elif month == 4 or month == 6 or month == 9 or month == 11:
      if day == 30:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
    elif month == 12:
      if day == 31:
        data2 = str(year+1) + '-01-01'
      else:
        data2 = x[:5] + '12-' + str(day + 1)
    else:
      if day == 31:
        data2 = x[:5] + str(month+1) + '-01'
      else:
        data2 = x[:5] + str(month) + '-' + str(day + 1)
  
  raw = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filterDate(lista_de_datas[dat], data2).filterBounds(area)
  ang_zenital = raw.aggregate_mean('SOLAR_ZENITH_ANGLE').getInfo()
  elevacao = 90 - ang_zenital
  cos_z = np.cos(ang_zenital*np.pi/180)
  dr = 1/((raw.aggregate_mean('EARTH_SUN_DISTANCE').getInfo())**2)

  print('This is the z angle: ', ang_zenital, 'This is the elevation: ', elevacao, 'This is the cos(Z): ', cos_z, 'This is the dr: ', dr)


  albedo_open = gdal.Open('albedo_2_' + lista_de_datas[dat] + '.tif')
  albedo_in_band = albedo_open.GetRasterBand(1)
  albedo_in_data = albedo_in_band.ReadAsArray().astype(np.float)

  reflectance_open = gdal.Open('reflectance_2_' + lista_de_datas[dat] + '.tif')
  NIR_in_band = reflectance_open.GetRasterBand(4)
  NIR_in_data = NIR_in_band.ReadAsArray().astype(np.float)

  RED_in_band = reflectance_open.GetRasterBand(3)
  RED_in_data = RED_in_band.ReadAsArray().astype(np.float)

  tsup_open = gdal.Open('tsup_2_' + lista_de_datas[dat] + '.tif')
  tsup_in_band = tsup_open.GetRasterBand(1)
  tsup_in_data = tsup_in_band.ReadAsArray().astype(np.float)

  nlcd_open = gdal.Open('mapbiomas_2015_2.tif')
  nlcd_in_band = nlcd_open.GetRasterBand(1)
  nlcd_in_data = nlcd_in_band.ReadAsArray().astype(np.float)

  data_nlcd.clear()

  albedo = np.ma.masked_where(albedo_in_data < 0, albedo_in_data)
  albedo = np.ma.masked_where(albedo > 1, albedo)
 
  NIR = np.where(NIR_in_data < 0.0, 0.0, NIR_in_data)
  NIR = np.ma.masked_where(NIR <= 0, NIR)
 
  RED = np.where(RED_in_data < 0.0, 0.0, RED_in_data)
  RED = np.ma.masked_where(RED <= 0, RED)
 
  tsup = np.ma.masked_where(tsup_in_data < 253, tsup_in_data)
  tsup = np.ma.masked_where(tsup > 340, tsup)
 
 
#EXPORTS#####################################################

  nome_ndvi = '_ndvi_landsat_col2.tif'
  export_ndvi = lista_de_datas[dat] + nome_ndvi
 
  nome_et24 = '_et_24_dms_landsat_2.tif'
  export_et24 = lista_de_datas[dat] + nome_et24

  nome_et24_alf = '_et_24_dms_col2_alf.tif'
  export_et24_alf = lista_de_datas[dat] + nome_et24_alf

  nome_et24_gr = '_et_24_dms_col2_gr.tif'
  export_et24_gr = lista_de_datas[dat] + nome_et24_gr

  nome_le = '_le_dms_landsat_col2.tif'
  export_le = lista_de_datas[dat] + nome_le

  nome_g = '_g_dms_landsat_col2.tif'
  export_g = lista_de_datas[dat] + nome_g

  nome_rn = '_rn_dms_landsat_col2.tif'
  export_rn = lista_de_datas[dat] + nome_rn 

  nome_h = '_h_dms_landsat_col2.tif'
  export_h = lista_de_datas[dat] + nome_h

  nome_fe = '_fe_dms_landsat_col2.tif'
  export_fe = lista_de_datas[dat] + nome_fe

  nome_tsup_q = '_tsup_pixls_quentes_col2.tif'
  export_tsup_q = lista_de_datas[dat] + nome_tsup_q

  nome_tsup_f = '_tsup_pixls_frios_col2.tif'
  export_tsup_f = lista_de_datas[dat] + nome_tsup_f

  nome_tsup = '_tsup_col2.tif'
  export_tsup = lista_de_datas[dat] + nome_tsup

  nome_albedo = '_albedo_col2.tif'
  export_albedo = lista_de_datas[dat] + nome_albedo

  nome_rah = '_rah_corr_col2.tif'
  export_rah = lista_de_datas[dat] + nome_rah

  nome_rn_24 = '_rn_24_col2.tif'
  export_rn_24 = lista_de_datas[dat] + nome_rn_24

#CALCULUS####################################################
  Tfac = 2.6 - (13*(lista_de_precipitation60[dat]/lista_de_ETr60[dat]))
  print(Tfac)

  savi = (1.1 * (NIR - RED)) / (0.1 + NIR + RED)

  iaf = np.where(savi < 0.688000, np.log((0.69 - savi) / 0.59) * (-1) / 0.91, 6)
  iafcorrigido = np.where(savi < 0.000001, 0.000001, iaf)
  iafcorrigido = np.where(iafcorrigido < 0.000001, 0.000001, iaf)

  e_o = np.where(iafcorrigido > 2.999999, 0.98, 0.95 + (0.01 * iafcorrigido))
  e_ocorrigido = np.where(iafcorrigido < 0.000001, 0.985, e_o)

  e_nb = np.where(iafcorrigido > 2.999999, 0.98, 0.97 + (0.0033 * iafcorrigido))
  e_nbcorrigido = np.where(iafcorrigido < 0.000001, 0.99, e_nb)

  rol_emi = e_o * 0.0000000567 * tsup**4

  e_a = 0.61078 * (np.exp((17.269 * lista_de_ta[dat])/(237.3 + lista_de_ta[dat]))) * lista_de_ur[dat]/100
  w = (10 * (0.14 * 10 * e_a * (lista_de_pa[dat]/101))) + 0.21
  transmitancia = 0.35 + 0.627 * (np.exp((0.00146*(-1)*lista_de_pa[dat]/cos_z) - 0.075 * ((w/cos_z)**0.4)))

  print ('e_a = : ', e_a)
  print ('w = : ', w)
  print ('transmitancia = : ', transmitancia)

  roc = 1367 * dr * cos_z * transmitancia
  print ('This is the shortwave radiation: ', roc)

  e_atm = 0.625 * (((1000 * e_a) / (273.15 + lista_de_ta[dat])) ** 0.131)

  rol_atm = e_atm * 0.0000000567 * (lista_de_ta[dat] + 273.15)**4
  print ('The Rad_atm is: ', rol_atm)

  rn = (1 - albedo) * roc - rol_emi + e_ocorrigido * rol_atm 

  ndvi = (NIR - RED) / (NIR + RED)

  g_corrigido = np.where(ndvi < 0.05, 0.3 * rn, ((tsup - 273.15) * (0.0038 + 0.0074 * albedo) * (1 - 0.98 * (ndvi ** 4))) * rn)

#//#//#//#//#//#//#//#//#//#// ENERGY BALANCE #//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//Cold Pixel - CIMEC

  p95 = np.percentile(ndvi[ndvi > 0], 95)
  print('This is the 95th percentile of the NDVI: ', p95)

  f1_frio_0 = np.ma.masked_where(nlcd_in_data != 36, ndvi)
  f1_frio = np.ma.masked_where(f1_frio_0 < p95, tsup)

  fff = f1_frio[f1_frio > 274]

  p20 =  np.percentile(fff[fff < 350], 20)
  print('This is the 20th percentile of Tsup: ', p20)

  f2_frio = np.ma.masked_where(f1_frio > p20, f1_frio)

  f2_frio_med = np.nanmean(f2_frio[f2_frio > 0])
  print('This is the average for the cool pixel: ', f2_frio_med)

  f3_frio = np.ma.masked_where(f2_frio > (f2_frio_med + 0.3), f2_frio)
  f3_frio = np.ma.masked_where(f2_frio < (f2_frio_med - 0.3), f3_frio)

  albedo_chave = 0.001343 * elevacao + (0.3281 * np.exp((-0.0188) * elevacao))
  print('The key-albedo is: ', albedo_chave)

  f4_frio = np.ma.masked_where(albedo > (albedo_chave + 0.02), f3_frio)
  f4_frio = np.ma.masked_where(albedo < (albedo_chave - 0.02), f4_frio)

  p_frio_ok = np.min(f4_frio[f4_frio > 0])

  print('This is the CIMEC cold pixel: ', p_frio_ok)

#//Hot Pixel - CIMEC
  p5 = np.percentile(ndvi[ndvi>0], 5)
  print('This is the 5th percentile of the NDVI: ', p5)
  f1_quente = np.ma.masked_where(ndvi > p5, tsup)

  f1_quente = np.ma.masked_where(f1_quente > 400, f1_quente)

  qqq = f1_quente[f1_quente > 274]

  p80 = np.percentile(qqq[qqq < 350], 80)
  print('This is the 80th percentile of the Tsup: ', p80)
  f2_quente = np.ma.masked_where(f1_quente < p80, f1_quente)


  f2_quente_med = np.nanmean(f2_quente)
  print('This is the average Ts of the pixels:', f2_quente_med)
  f2_quente_med_2 = f2_quente_med - Tfac
  print('This is the average of the Ts of the pixels - Tfac: ', f2_quente_med_2)

  f3_quente = np.ma.masked_where(f2_quente > (f2_quente_med_2 + 2.0), f2_quente)
  f3_quente = np.ma.masked_where(f3_quente < (f2_quente_med_2 - 2.0), f3_quente)

  p_quente_ok = np.max(f3_quente[f3_quente > 0])
#p_quente_ok = f3_quente.max()

  print('This is CIMEC hot pixel: ', p_quente_ok)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///

#//SAVI OF THE HOT PIXEL
  savi_pq = np.ma.masked_where(ndvi > p5, tsup)
  savi_pq = np.ma.masked_where(savi_pq != p_quente_ok, savi)
  savi_quente = savi_pq.max()
  print ('The SAVI of the hot pixel is: ', savi_quente)

#//Rn OF THE HOT PIXEL
  rn_pq = np.ma.masked_where(ndvi > p5, tsup)
  rn_pq = np.ma.masked_where(rn_pq != p_quente_ok, rn)
  rn_quente = rn_pq.max()
  print ('The Rn of the hot pixel is: ', rn_quente)

#//G OF THE HOT PIXEL
  g_pq = np.ma.masked_where(ndvi > p5, tsup)
  g_pq = np.ma.masked_where(g_pq != p_quente_ok, g_corrigido)
  g_quente = g_pq.max()
  print ('The G of the hot pixel is: ', g_quente)

#//calculating linear regression coefficients
  u_estrela1 = lista_de_u[dat] * 0.41/np.log(10/0.024)# Attention in this value 10 here in the logarithm! 10 stands for wind measurement height (10 m)
  u200 = u_estrela1 * np.log(200/0.024)/0.41
  zom = np.exp(-5.809+5.62*savi_quente)
  u_estrela = (u200*0.41)/np.log(200/zom)
  rah = (np.log(2/0.1))/(0.41 * u_estrela)
  b = (rn_quente - g_quente)*rah / (1.15*1004*(p_quente_ok - p_frio_ok))
  

  a = (-1)*b*(p_frio_ok - 273.15)
  dT = a + b *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a + (b * (p_quente_ok - 273.15)))/rah

  print ('The sensible heat flux H is: ', H)
  rn_g = rn_quente - g_quente
  print ('Rn - G is: ', rn_g)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculating in the images#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  zom_img = np.exp(-5.809+5.62*savi)
  u_estrela_img = (u200*0.41)/np.log(200/zom_img)
  rah_img = (np.log(2/0.1))/(0.41 * u_estrela_img)
  H_img = 1.15 * 1004 * (a + (b * (tsup - 273.15)))/rah_img

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//START OF THE ITERATIVE PROCESS#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//1st CORRECTION#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
  L = (-1)*1.15*1004*(u_estrela**3)*p_quente_ok/(0.41*9.81*H)
  if L <= 0.0:
    x_0_1m = ((1-16)*0.1/L)**0.25
    x_2m = ((1-16)*2/L)**0.25
    x_200m = ((1-16)*200/L)**0.25

    psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
    psi_h2 = 2*np.log((1+(x_2m**2))/2)
    psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi
  else:
    psi_h0_1 = (-5) * (0.1/L)
    psi_h2 = (-5) * (2/L)
    psi_m200 = (-5) * (200/L)

  u_estrela_corr_1 = u200*0.41/((np.log(200/zom))-psi_m200)
  rah_corr_1 = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*u_estrela_corr_1)
  b_corr = (rn_quente - g_quente)*rah_corr_1 / (1.15*1004*(p_quente_ok - p_frio_ok))
  a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
  print('This is the a_corr: ', a_corr, 'This is the b_corr: ', b_corr)
  dT_corr = a_corr + b_corr *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/rah_corr_1

  print ('The sensible heat flux H is: ', H)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculando 1ª CORREÇÃO nas imagens #//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  L_img = (-1)*1.15*1004*(u_estrela_img**3)*tsup/(0.41*9.81*H_img)
  
  x_0_1m_img = ((1-16)*0.1/L_img)**0.25
  x_2m_img = ((1-16)*2/L_img)**0.25
  x_200m_img = ((1-16)*200/L_img)**0.25

  psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
  psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
  psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                          (-1)*5*(200/L_img))
  u_estrela_corr_img_1 = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
  rah_corr_img_1 = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * u_estrela_corr_img_1)
  H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/rah_corr_img_1

  err_rel = abs(rah_corr_1 - rah)*100/rah
  print('a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)
  print('H is: ', H)

  lista_u_star = []
  lista_rah = []
  lista_u_star_img = []
  lista_rah_img = []

  lista_u_star.append(u_estrela_corr_1)
  
  lista_rah.append(rah_corr_1)
    lista_u_star_img.append(u_estrela_corr_img_1)
  lista_rah_img.append(rah_corr_img_1)


  rodadas = 1

  while err_rel > 1.5:
    rodadas += 1
    lista_u_star.append('u_estrela_corr_{}'.format(rodadas))
    
    lista_rah.append('rah_corr_{}'.format(rodadas))
    
    lista_u_star_img.append('u_estrela_corr_img_{}'.format(rodadas))    
    lista_rah_img.append('rah_corr_img_{}'.format(rodadas))
 
    L = (-1)*1.15*1004*((lista_u_star[0])**3)*p_quente_ok/(0.41*9.81*H)
 
    if L <= 0.0:
      x_0_1m = ((1-16)*0.1/L)**0.25      
      x_2m = ((1-16)*2/L)**0.25      
      x_200m = ((1-16)*200/L)**0.25      
 
      psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)      
      psi_h2 = 2*np.log((1+(x_2m**2))/2)      
      psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi      
    else:
      psi_h0_1 = (-5) * (0.1/L)
      psi_h2 = (-5) * (2/L)
      psi_m200 = (-5) * (200/L)
    
    
    lista_u_star[1] = u200*0.41/((np.log(200/zom))-psi_m200)
    lista_rah[1] = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*(lista_u_star[1]))

    b_corr = (rn_quente - g_quente)*lista_rah[1] / (1.15*1004*(p_quente_ok - p_frio_ok))
    
    a_corr = (-1)*b_corr*(p_frio_ok - 273.15)

    H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/lista_rah[1]

    #calculating in the images:
    L_img = (-1)*1.15*1004*(((lista_u_star_img[0]))**3)*tsup/(0.41*9.81*H_corr_img)
    x_0_1m_img = ((1-16)*0.1/L_img)**0.25
    x_2m_img = ((1-16)*2/L_img)**0.25
    x_200m_img = ((1-16)*200/L_img)**0.25
 
    psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
    psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
    psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                              (-1)*5*(200/L_img))
    lista_u_star_img[1] = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
    lista_rah_img[1] = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * lista_u_star_img[1])
    H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/lista_rah_img[1]
       
    
    err_rel = abs(lista_rah[1] - lista_rah[0])*100/lista_rah[1]
    print('Round',rodadas,' a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)
    print ('H is : ', H)

    #excluíndo o primeiro ítem de cada lista
    lista_u_star.pop(0)
    lista_rah.pop(0)
    lista_u_star_img.pop(0)
    lista_rah_img.pop(0)

  print('End of the process, a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)

#//Calculando o Fluxo de calor latente (LE) e a ET_24h

  LE = rn - g_corrigido - H_corr_img
  LE = np.where(LE < 0.0, 0.0, LE)
  fe = LE / (rn - g_corrigido)

  del_ = 0.006918-0.399912*np.cos(2*np.pi*(lista_de_DOA[dat]/365))+0.070257*np.sin(2*np.pi*(lista_de_DOA[dat]/365))-0.006758*np.cos(2*2*np.pi*(lista_de_DOA[dat]/365))+0.000907*np.sin(2*2*np.pi*(lista_de_DOA[dat]/365))-0.002697*np.cos(3*2*np.pi*(lista_de_DOA[dat]/365))+0.00148*np.sin(3*2*np.pi*(lista_de_DOA[dat]/365))
  fi = lat * np.pi/180
  H_hor = np.arccos(-np.tan(fi)*np.tan(del_))
  rs_24_toa = 118.08*dr*(H_hor*np.sin(fi)*np.sin(del_)+np.cos(fi)*np.cos(del_)*np.sin(H_hor))/np.pi
  print('Rs_TOA is: ', rs_24_toa)

  tsw_24 = lista_de_rs_24[dat] / rs_24_toa
  print('tsw_24 is: ', tsw_24)
  rn_24 = (1-albedo)*lista_de_rs_24[dat]*11.57407 - 110*tsw_24 #//Rs_24 transformado em W/m²


  ET_24 = 0.0353*fe*rn_24
  ET_24 = np.ma.masked_where(ET_24 < 0, ET_24)
  ET_24 = ET_24.filled(-99)

  print('#'*150)
  print('The image(s) of the date {} is(are) beeing exported'.format(lista_de_datas[dat]))
  print('#'*150)


  out_ds = make_raster(albedo_open, export_et24, ET_24, gdal.GDT_Float32, -99)

del out_ds

# Exporting MODIS images to your drive

In [ ]:
# Define your study area
'''mandacaru = ee.Geometry.Polygon(
        [[[-40.721, -9.282],
          [-40.174, -9.282],
          [-40.174, -9.631],
          [-40.721, -9.631]]])
'''
mandacaru = ee.Geometry.Polygon(                        
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

'''mandacaru = ee.Geometry.Polygon(
        [[[-40.443765656859135, -9.369681936878271],
          [-40.443765656859135, -9.4184573955032],
          [-40.38179589977906, -9.4184573955032],
          [-40.38179589977906, -9.369681936878271]]])'''

#Choose the dates of the MODIS images that will be processed in the STARFM algorithm
#lista_de_datas_terra = ['2015_07_31', '2015_11_20', '2015_12_06', '2016_03_11', '2016_04_28', '2016_05_14', '2015_06_16', '2015_07_19', '2015_07_22', '2015_08_20', '2015_08_21', '2015_08_25', '2015_08_28', '2015_08_29', '2015_09_08', '2015_09_10', '2015_09_12', '2015_09_29', '2015_10_05', '2015_10_17', '2015_10_21', '2015_10_28', '2015_11_06', '2015_11_13', '2015_11_14', '2015_11_15', '2015_11_17', '2015_12_01', '2015_12_04', '2015_12_08', '2015_12_11', '2015_12_15', '2016_01_12', '2016_02_29', '2016_03_06', '2016_04_09', '2016_04_26', '2016_05_18']
lista_de_datas_aqua = ['2015_08_24', '2015_09_25', '2015_10_03', '2015_10_27', '2015_11_12', '2015_12_14', '2016_05_22', '2015_07_29', '2015_08_05', '2015_09_02', '2015_09_04', '2015_09_06', '2015_09_11', '2015_09_13', '2015_09_22', '2015_09_24', '2015_09_26', '2015_10_02', '2015_10_04', '2015_10_10', '2015_10_13', '2015_10_18', '2015_10_22', '2015_10_26', '2015_10_29', '2015_11_11', '2015_11_30', '2015_12_02', '2015_12_13', '2016_02_08', '2016_03_07']

produto_1 = ee.String('modis_gq')
produto_2 = ee.String('modis_a1')
produto_3 = ee.String('modis_a3')

#numero_de_datas_terra = len(lista_de_datas_terra)
numero_de_datas_aqua = len(lista_de_datas_aqua)

nome_gq = ee.String('MODIS/006/MYD09GQ/')
nome_a1 = ee.String('MODIS/006/MYD11A1/')
nome_a3 = ee.String('MODIS/006/MCD43A3/')

#for dat in range(0, lista_de_datas_terra):
for dat in range(0, numero_de_datas_aqua):
  #data = ee.String(lista_de_datas_terra[dat])
  data = ee.String(lista_de_datas_aqua[dat])
  pegar_gq_1 = nome_gq.cat(data)
  pegar_a1_1 = nome_a1.cat(data)
  pegar_a3_1 = nome_a3.cat(data)

  img1 = ee.Image(pegar_gq_1.getInfo())
  img4 = ee.Image(pegar_a1_1.getInfo())
  img7 = ee.Image(pegar_a3_1.getInfo())

  img_export1 = img1.select(['sur_refl_b01', 'sur_refl_b02']).multiply(0.0001)
  img_export4 = img4.select(['LST_Day_1km']).multiply(0.02)
  img_export7 = img7.select(['Albedo_BSA_shortwave']).multiply(0.001)

  saida_1 = produto_1.cat(data)
  saida_4 = produto_2.cat(data)
  saida_7 = produto_3.cat(data)


  task1 = ee.batch.Export.image.toDrive(image=img_export1, folder='Pet_Juaz', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_1.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  task4 = ee.batch.Export.image.toDrive(image=img_export4, folder='Pet_Juaz', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_4.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')
  task7 = ee.batch.Export.image.toDrive(image=img_export7, folder='Pet_Juaz', description='modis', scale=30, region=mandacaru, fileNamePrefix=saida_7.getInfo(), crs='EPSG:32724', fileFormat='GeoTIFF')

  task1.start()
  task4.start()
  task7.start()

print('End of the Script')

# STARFM dois pares de imagens

In [ ]:
def make_slices(data, win_size):
    """Return a list of slices given a window size.
    data     - two-dimensional array to get slices from
    win_size - tuple of (rows, columns) for the moving window"""
    rows = data.shape[0] - win_size[0] + 1
    cols = data.shape[1] - win_size[1] + 1
    slices = []
    for i in range(win_size[0]):
        for j in range(win_size[1]):
            slices.append(data[i:rows+i, j:cols+j])
    return slices

# This script selects the Spectrally Similar Neighbor Pixels and exports them with Euclidean distance information.
import time
start_time = time.time()
# Define the directory that has the images to be processed.
os.chdir(r"/content/drive/MyDrive/colab_imagens/Pet_Juaz")


lista_de_datas_landsat_antes = ['2015-10-03', '2015-10-03', '2015-10-03']#Landsat first dates
lista_de_datas_landsat_depois = ['2015-10-27', '2015-10-27', '2015-10-27']

lista_de_datas_modis_antes = ['2015_10_03', '2015_10_03', '2015_10_03']
lista_de_datas_modis_depois = ['2015_10_27', '2015_10_27', '2015_10_27']

lista_de_datas_modis_to = ['2015_10_21', '2015_10_22', '2015_10_26']


numero_de_datas = len(lista_de_datas_modis_to)


for dat in range(0, numero_de_datas):
  #Reflectances
  in_ds = gdal.Open('reflectance_' + lista_de_datas_landsat_antes[dat] + '.tif')
  in_band_lv = in_ds.GetRasterBand(3) # Get the red band
  in_band_liv = in_ds.GetRasterBand(4)  # Get the IR band

  in_ds_mv = gdal.Open('modis_gq' + lista_de_datas_modis_antes[dat] + '.tif') # Load the first MODIS image (the day before the predicted date)
  in_band_mv = in_ds_mv.GetRasterBand(1) # Get the red band
  in_band_miv = in_ds_mv.GetRasterBand(2) # Get the IR band

  in_ds_k2 = gdal.Open('reflectance_' + lista_de_datas_landsat_depois[dat] + '.tif')# Load the second LANDSAT image (the day after the predicted date)
  in_band_lv_k2 = in_ds_k2.GetRasterBand(3) # Get the red band
  in_band_liv_k2 = in_ds_k2.GetRasterBand(4) # Get the red band

  in_ds_mv_k2 = gdal.Open('modis_gq' + lista_de_datas_modis_depois[dat] + '.tif') # Load the second MODIS image (the day after the predicted date)
  in_band_mv_k2 = in_ds_mv_k2.GetRasterBand(1) # Get the red band
  in_band_miv_k2 = in_ds_mv_k2.GetRasterBand(2) # Get the IR band

  in_ds_mv_ko = gdal.Open('modis_gq' + lista_de_datas_modis_to[dat] + '.tif') # Load the third MODIS image (predicted date)
  in_band_mv_ko = in_ds_mv_ko.GetRasterBand(1) # Get the red band
  in_band_miv_ko = in_ds_mv_ko.GetRasterBand(2) # Get the IR band

#Albedo (we don't need it)
  open_l_albedo_1 = gdal.Open('albedo_' + lista_de_datas_landsat_antes[dat] + '.tif')
  gb_l_albedo_1 = open_l_albedo_1.GetRasterBand(1)

  open_m_albedo_1 = gdal.Open('modis_a3' + lista_de_datas_modis_antes[dat] + '.tif')
  gb_m_albedo_1 = open_m_albedo_1.GetRasterBand(1)

  open_l_albedo_2 = gdal.Open('albedo_' + lista_de_datas_landsat_depois[dat] + '.tif')
  gb_l_albedo_2 = open_l_albedo_2.GetRasterBand(1)

  open_m_albedo_2 = gdal.Open('modis_a3' + lista_de_datas_modis_depois[dat] + '.tif')
  gb_m_albedo_2 = open_m_albedo_2.GetRasterBand(1)

  open_m_albedo_0 = gdal.Open('modis_a3' + lista_de_datas_modis_to[dat] + '.tif')
  gb_m_albedo_0 = open_m_albedo_0.GetRasterBand(1)

#Land Surface Temperature (we don't need it)
  open_l_tsup_1 = gdal.Open('tsup_' + lista_de_datas_landsat_antes[dat] + '.tif')
  gb_l_tsup_1 = open_l_tsup_1.GetRasterBand(1)

  open_m_tsup_1 = gdal.Open('modis_a1' + lista_de_datas_modis_antes[dat] + '.tif')
  gb_m_tsup_1 = open_m_tsup_1.GetRasterBand(1)

  open_l_tsup_2 = gdal.Open('tsup_' + lista_de_datas_landsat_depois[dat] + '.tif')
  gb_l_tsup_2 = open_l_tsup_2.GetRasterBand(1)

  open_m_tsup_2 = gdal.Open('modis_a1' + lista_de_datas_modis_depois[dat] + '.tif')
  gb_m_tsup_2 = open_m_tsup_2.GetRasterBand(1)

  open_m_tsup_0 = gdal.Open('modis_a1' + lista_de_datas_modis_to[dat] + '.tif')
  gb_m_tsup_0 = open_m_tsup_0.GetRasterBand(1)


#Outputs
  var = 'ref_verm_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  name = var + lista_de_datas_modis_to[dat] + '.tif'

  variv = 'ref_iv_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  nameiv = variv + lista_de_datas_modis_to[dat] + '.tif'

  varalbedo = 'albedo_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  namealbedo = varalbedo + lista_de_datas_modis_to[dat] + '.tif'

  vartsup = 'tsup_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  nametsup = vartsup + lista_de_datas_modis_to[dat] + '.tif'

  varndvi = 'ndvi_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  namendvi = varndvi + lista_de_datas_modis_to[dat] + '.tif'

  varsavi = 'savi_' #examples: albedo_, ref_iv_, ref_verm_ and tsup_
  namesavi = varsavi + lista_de_datas_modis_to[dat] + '.tif'


  xsize = in_band_lv.XSize #number of columns
  ysize = in_band_lv.YSize #number of lines

  driver = gdal.GetDriverByName('GTiff')  

  mw = 49 #moving window
  lp = mw - 1 #number of lines lost.
  ns = mw * mw #number of slices
  lsp = int(lp / 2) #number of top rows lost
  sc = ns // 2 #central slice
  n = 2 #number of lines to resolve

  out_ds_lv = driver.Create(name, xsize, ysize, 1, gdal.GDT_Float32) #creating the output image
  out_ds_liv = driver.Create(nameiv, xsize, ysize, 1, gdal.GDT_Float32)
  out_ds_albedo = driver.Create(namealbedo, xsize, ysize, 1, gdal.GDT_Float32)
  out_ds_tsup = driver.Create(nametsup, xsize, ysize, 1, gdal.GDT_Float32)

  out_ds_lv.SetProjection(in_ds.GetProjection()) #assigning the same projection as the input image, EPSG 32724
  out_ds_liv.SetProjection(in_ds.GetProjection()) #assigning the same projection as the input image, EPSG 32724
  out_ds_albedo.SetProjection(in_ds.GetProjection())
  out_ds_tsup.SetProjection(in_ds.GetProjection())

  out_ds_lv.SetGeoTransform(in_ds.GetGeoTransform()) #applying the new GeoTransform to the output image
  out_ds_liv.SetGeoTransform(in_ds.GetGeoTransform())
  out_ds_albedo.SetGeoTransform(in_ds.GetGeoTransform())
  out_ds_tsup.SetGeoTransform(in_ds.GetGeoTransform())

  out_band_lv = out_ds_lv.GetRasterBand(1)
  out_band_lv.SetNoDataValue(-99)

  out_band_liv = out_ds_liv.GetRasterBand(1)
  out_band_liv.SetNoDataValue(-99)

  out_band_albedo = out_ds_albedo.GetRasterBand(1)
  out_band_albedo.SetNoDataValue(-99)

  out_band_tsup = out_ds_tsup.GetRasterBand(1)
  out_band_tsup.SetNoDataValue(-99)

  for i in range(0, ysize, n): #loop start to resolve the image using chunks.
      if n + lp + i < ysize: #the number of lines to be read () + the number of lines read previously must be less than the total number of lines
          rows = n + lp #for a 49x49 moving window, 48 lines are lost, so, to solve 2 lines, you should read 50
      else:
          rows = ysize - i
          if rows < mw: #if the number of lines to be read is less than the moving window (49) the script must stop.
              break
      yoff = max(0, i - 1)

      in_data_lv = in_band_lv.ReadAsArray(0, i, xsize, rows).astype(float) #reading from 1st column and i-row all col and a number of rows = rows
      in_data_mv = in_band_mv.ReadAsArray(0, i, xsize, rows).astype(float)
      in_data_lv_k2 = in_band_lv_k2.ReadAsArray(0, i, xsize, rows).astype(float)
      in_data_mv_k2 = in_band_mv_k2.ReadAsArray(0, i, xsize, rows).astype(float)
      in_data_mv_ko = in_band_mv_ko.ReadAsArray(0, i, xsize, rows).astype(float)

      in_data_liv = in_band_liv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      in_data_miv = in_band_miv.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      in_data_liv_k2 = in_band_liv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      in_data_miv_k2 = in_band_miv_k2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      in_data_miv_ko = in_band_miv_ko.ReadAsArray(0, i, xsize, rows).astype(np.float32)

      id_l_albedo_1 = gb_l_albedo_1.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_albedo_1 = gb_m_albedo_1.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_l_albedo_2 = gb_l_albedo_2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_albedo_2 = gb_m_albedo_2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_albedo_0 = gb_m_albedo_0.ReadAsArray(0, i, xsize, rows).astype(np.float32)

      id_l_tsup_1 = gb_l_tsup_1.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_tsup_1 = gb_m_tsup_1.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_l_tsup_2 = gb_l_tsup_2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_tsup_2 = gb_m_tsup_2.ReadAsArray(0, i, xsize, rows).astype(np.float32)
      id_m_tsup_0 = gb_m_tsup_0.ReadAsArray(0, i, xsize, rows).astype(np.float32)
    
#Creating the Slices
      slices_lv = make_slices(in_data_lv, (mw, mw)) #creating the 2401 slices for the Lansat and MODIS red band
      slices_mv = make_slices(in_data_mv, (mw, mw))
      slices_lv_k2 = make_slices(in_data_lv_k2, (mw, mw))
      slices_mv_k2 = make_slices(in_data_mv_k2, (mw, mw))
      slices_mv_ko = make_slices(in_data_mv_ko, (mw, mw))

      difabsv = [abs(valor - slices_lv[sc]) for valor in slices_lv] #using list comprehension to subtract from each slice the central slice
      desvpadv = np.std(np.ma.dstack(slices_lv), 2) #taking the standard deviation in the z dimension, that is, the deviation of all pixels in the moving window
      menorqdesvv = np.ma.masked_where(difabsv > desvpadv, slices_lv) #masking pixels where the absolute difference is greater than the standard deviation

      difabsv_k2 = [abs(valor - slices_lv_k2[sc]) for valor in slices_lv_k2]
      desvpadv_k2 = np.std(np.ma.dstack(slices_lv_k2), 2)
      menorqdesvv_k2 = np.ma.masked_where(difabsv_k2 > desvpadv_k2, slices_lv_k2)

      filtro_lv_k1 = np.ma.masked_where(menorqdesvv + menorqdesvv_k2 == -1, menorqdesvv) #no sum will be =-1, so the value of menorqdesvv will be preserved, but only for pixels that are neighbors for both variables
      filtro_lv_k2 = np.ma.masked_where(menorqdesvv + menorqdesvv_k2 == -1, menorqdesvv_k2)
# end of 3rd step

      maxsijk = np.where(abs(slices_lv[sc] - slices_mv[sc]) > abs(slices_lv_k2[sc] - slices_mv_k2[sc]),
                    abs(slices_lv[sc] - slices_mv[sc]), abs(slices_lv_k2[sc] - slices_mv_k2[sc])) #creating the matrix that presents the maximum Sijk between t1 and t2

      sijk1 = abs(filtro_lv_k1 - slices_mv) #creating Sijk just for neighbors
      sijk1_filt = np.ma.masked_where(sijk1 > maxsijk + 0.01, filtro_lv_k1)
      sijk2 = abs(filtro_lv_k2 - slices_mv_k2)
      sijk2_filt = np.ma.masked_where(sijk2 > maxsijk + 0.01, filtro_lv_k2)

      sijk1_final = np.ma.masked_where(sijk1_filt + sijk2_filt == -1, sijk1_filt) #selecting only the pixels that obey the inequality in t1 and t2

      maxtijk = np.where(abs(slices_mv[sc] - slices_mv_ko[sc]) > abs(slices_mv_k2[sc] - slices_mv_ko[sc]),
                    abs(slices_mv[sc] - slices_mv_ko[sc]), abs(slices_mv_k2[sc] - slices_mv_ko[sc])) #repeating the process for Tijk

      filtro_mv_ko = np.ma.masked_where(slices_mv_ko + sijk1_final == -1, slices_mv_ko)

      tijk1 = abs(slices_mv - filtro_mv_ko)
      tijk1_filt = np.ma.masked_where(tijk1 > maxtijk + 0.01, slices_mv)
      tijk2 = abs(slices_mv_k2 - filtro_mv_ko)
      tijk2_filt = np.ma.masked_where(tijk2 > maxtijk + 0.01, slices_mv_k2)

      tijk1_final = np.ma.masked_where(tijk1_filt + tijk2_filt == -1, tijk1_filt)

      neighbours = np.ma.masked_where(tijk1_final + sijk1_final == -1, sijk1_final) #defining neighbors according to Sijk and Tijk
# end of step 4 for red band
#repeating the steps for Landsat and MODIS IR band
      slices_liv = make_slices(in_data_liv, (mw, mw))
      slices_miv = make_slices(in_data_miv, (mw, mw))
      slices_liv_k2 = make_slices(in_data_liv_k2, (mw, mw))
      slices_miv_k2 = make_slices(in_data_miv_k2, (mw, mw))
      slices_miv_ko = make_slices(in_data_miv_ko, (mw, mw))

      difabsiv = [abs(valoriv - slices_liv[sc]) for valoriv in slices_liv]
      desvpadiv = np.std(np.ma.dstack(slices_liv), 2)
      menorqdesviv = np.ma.masked_where(difabsiv > desvpadiv, slices_liv)

      difabsiv_k2 = [abs(valoriv - slices_liv_k2[sc]) for valoriv in slices_liv_k2]
      desvpadiv_k2 = np.std(np.ma.dstack(slices_liv_k2), 2)
      menorqdesviv_k2 = np.ma.masked_where(difabsiv_k2 > desvpadiv_k2, slices_liv_k2)

      filtro_liv_k1 = np.ma.masked_where(menorqdesviv + menorqdesviv_k2 == -1, menorqdesviv)
      filtro_liv_k2 = np.ma.masked_where(menorqdesviv + menorqdesviv_k2 == -1, menorqdesviv_k2)
        # end of step 3 for IR

      maxsijkiv = np.where(abs(slices_liv[sc] - slices_miv[sc]) > abs(slices_liv_k2[sc] - slices_miv_k2[sc]),
                       abs(slices_liv[sc] - slices_miv[sc]), abs(slices_liv_k2[sc] - slices_miv_k2[sc]))

      sijk1iv = abs(filtro_liv_k1 - slices_miv)
      sijk1iv_filt = np.ma.masked_where(sijk1iv > maxsijkiv + 0.015, filtro_liv_k1)
      sijk2iv = abs(filtro_liv_k2 - slices_miv_k2)
      sijk2iv_filt = np.ma.masked_where(sijk2iv > maxsijkiv + 0.015, filtro_liv_k2)

      sijk1iv_final = np.ma.masked_where(sijk1iv_filt + sijk2iv_filt == -1, sijk1iv_filt)

      maxtijkiv = np.where(abs(slices_miv[sc] - slices_miv_ko[sc]) > abs(slices_miv_k2[sc] - slices_miv_ko[sc]),
                       abs(slices_miv[sc] - slices_miv_ko[sc]), abs(slices_miv_k2[sc] - slices_miv_ko[sc]))

      filtro_miv_ko = np.ma.masked_where(slices_miv_ko + sijk1iv_final == -1, slices_miv_ko)

      tijk1iv = abs(slices_miv - filtro_miv_ko)
      tijk1iv_filt = np.ma.masked_where(tijk1iv > maxtijkiv + 0.015, slices_miv)
      tijk2iv = abs(slices_miv_k2 - filtro_miv_ko)
      tijk2iv_filt = np.ma.masked_where(tijk2iv > maxtijkiv + 0.015, slices_miv_k2)

      tijk1iv_final = np.ma.masked_where(tijk1iv_filt + tijk2iv_filt == -1, tijk1iv_filt)

      neighboursiv = np.ma.masked_where(tijk1iv_final + sijk1iv_final == -1, sijk1iv_final)
# end of step 4 for IR band

      neighbours = np.ma.masked_where(neighbours + neighboursiv == -1, neighbours) #ddefining neighbors according to R and IR

      for s in range(0, ns): #creating a loop between layers of neighbors

          y = (s + 1) // (mw + 1) #calculating the y-coordinate of each pixel of the current s-layer
          x = s + 1 - (y * mw) - 1 #calculating the x-coordinate of each pixel of the current s-layer
          D = 1 + ((np.sqrt(np.square(x - lsp) + np.square(y - lsp))) * 30) / 250 #calculating D of each pixel of the current s-layer

          neighbours[s] = np.where(neighbours[s] > 0, D, -99)

# STARFM EXECUTION FOR RED BAND
      sijk1 = np.ma.masked_where(neighbours < 0.0, sijk1)
      tijk1 = np.ma.masked_where(neighbours < 0.0, tijk1)
      sijk2 = np.ma.masked_where(neighbours < 0.0, sijk2)
      tijk2 = np.ma.masked_where(neighbours < 0.0, tijk2)

      Cijk1 = neighbours * np.log((sijk1 * 10000) + 1) * np.log((tijk1 * 10000) + 1)# sijk1 and tijk1 are not filtered but the neighbors (distance info) are
      Cijk2 = neighbours * np.log((sijk2 * 10000) + 1) * np.log((tijk2 * 10000) + 1)
        
      divisao1 = np.where(Cijk1 == 0.0, 0, 1 / Cijk1)
      divisao2 = np.where(Cijk2 == 0.0, 0, 1 / Cijk2)

      Wijk1 = divisao1 / (divisao1.sum(0) + divisao2.sum(0)) #in the .sum(0) function the sum is vertical (the numpy array has the format z, y, x)
      Wijk2 = divisao2 / (divisao1.sum(0) + divisao2.sum(0)) #(z, y, x) -> (0, 1, 2), so you should use .sum(0)       
        
      M_L_M_1= slices_mv_ko[sc] + slices_lv[sc] - slices_mv[sc]      #M(xw/2, yw/2, to) + L(xw/2, yw/2, tk1) - M(xw/2, yw/2, tk1)
      M_L_M_2 = slices_mv_ko[sc] + slices_lv_k2[sc] - slices_mv_k2[sc]#M(xw/2, yw/2, to) + L(xw/2, yw/2, tk2) - M(xw/2, yw/2, tk2)
        
      if n + lp + i <= ysize:
        L = np.zeros((n, xsize-lp), np.float) #creating an array with values 0, 2 rows and xsize columns
      else:
        linhas = (ysize - lp) - (((ysize - lp)//n)*n)
        L = np.zeros((linhas, xsize-lp), np.float)
      for m in range(0, ns): #creating a loop inside the number of Wijk layers
        L = L + ((Wijk1[m] * M_L_M_1) + (Wijk2[m] * M_L_M_2)) #L initially is 0, but it is replaced as the sums are carried out

      L = np.where(tijk1[sc] == 0.0, M_L_M_1, L)# Filter for when there is no difference between MODIS imgs (01 and 00)
      L = np.where(sijk1[sc] == 0.0, M_L_M_1, L)# Filter for when there is no difference between L-M imgs (01)
      L = np.where(tijk2[sc] == 0.0, M_L_M_2, L)# Filter for when there is no difference between MODIS imgs (02 and 00)
      L = np.where(sijk1[sc] == 0.0, M_L_M_2, L)# Filter for when there is no difference between L-M imgs (02)

      out_data_lv = np.ones(in_data_lv.shape, np.float32) * -99 #creating an array in read array format (chunk)
      out_data_lv[lsp:-lsp, lsp:-lsp] = L #taking the 1st array from the neighbors list and inserting it into the newly created array
        
      if i == 0:
        out_band_lv.WriteArray(out_data_lv)
      else:
        out_band_lv.WriteArray(out_data_lv[lsp:], 0, i + lsp)


# EXECUTION OF STARFM FOR IR
      sijk1iv = np.ma.masked_where(neighbours < 0.0, sijk1iv)
      tijk1iv = np.ma.masked_where(neighbours < 0.0, tijk1iv)
      sijk2iv = np.ma.masked_where(neighbours < 0.0, sijk2iv)
      tijk2iv = np.ma.masked_where(neighbours < 0.0, tijk2iv)

      Cijk1iv = neighbours * np.log((sijk1iv * 10000) + 1) * np.log((tijk1iv * 10000) + 1)
      Cijk2iv = neighbours * np.log((sijk2iv * 10000) + 1) * np.log((tijk2iv * 10000) + 1)
        
      divisao1iv = np.where(Cijk1iv == 0.0, 0, 1 / Cijk1iv)
      divisao2iv = np.where(Cijk2iv == 0.0, 0, 1 / Cijk2iv)

      Wijk1iv = divisao1iv / (divisao1iv.sum(0) + divisao2iv.sum(0))
      Wijk2iv = divisao2iv / (divisao1iv.sum(0) + divisao2iv.sum(0))       
        
      M_L_M_1iv= slices_miv_ko[sc] + slices_liv[sc] - slices_miv[sc]
      M_L_M_2iv = slices_miv_ko[sc] + slices_liv_k2[sc] - slices_miv_k2[sc]
        
      if n + lp + i <= ysize:
        Liv = np.zeros((n, xsize-lp), np.float)
      else:
        linhas = (ysize - lp) - (((ysize - lp)//n)*n)
        Liv = np.zeros((linhas, xsize-lp), np.float)
      for m in range(0, ns):
          Liv = Liv + ((Wijk1iv[m] * M_L_M_1iv) + (Wijk2iv[m] * M_L_M_2iv))

      Liv = np.where(tijk1iv[sc] == 0.0, M_L_M_1iv, Liv)
      Liv = np.where(sijk1iv[sc] == 0.0, M_L_M_1iv, Liv)
      Liv = np.where(tijk2iv[sc] == 0.0, M_L_M_2iv, Liv)
      Liv = np.where(sijk1iv[sc] == 0.0, M_L_M_2iv, Liv)

      out_data_liv = np.ones(in_data_lv.shape, np.float32) * -99
      out_data_liv[lsp:-lsp, lsp:-lsp] = Liv

      if i == 0:
        out_band_liv.WriteArray(out_data_liv)
      else:
        out_band_liv.WriteArray(out_data_liv[lsp:], 0, i + lsp)

#Creating Slices for Albedo
      slices_l_albedo_1 = make_slices(id_l_albedo_1, (mw, mw))
      slices_m_albedo_1 = make_slices(id_m_albedo_1, (mw, mw))
      slices_l_albedo_2 = make_slices(id_l_albedo_2, (mw, mw))
      slices_m_albedo_2 = make_slices(id_m_albedo_2, (mw, mw))
      slices_m_albedo_0 = make_slices(id_m_albedo_0, (mw, mw))

      slices_l_albedo_1 = np.ma.masked_where(neighbours < 0.0, slices_l_albedo_1)
      slices_m_albedo_1 = np.ma.masked_where(neighbours < 0.0, slices_m_albedo_1)
      slices_l_albedo_2 = np.ma.masked_where(neighbours < 0.0, slices_l_albedo_2)
      slices_m_albedo_2 = np.ma.masked_where(neighbours < 0.0, slices_m_albedo_2)
      slices_m_albedo_0 = np.ma.masked_where(neighbours < 0.0, slices_m_albedo_0)

# RUNNING STARFM FOR Albedo
      sijk1albedo = abs(slices_l_albedo_1 - slices_m_albedo_1)
      tijk1albedo = abs(slices_m_albedo_1 - slices_m_albedo_0)
      sijk2albedo = abs(slices_l_albedo_2 - slices_m_albedo_2)
      tijk2albedo = abs(slices_m_albedo_2 - slices_m_albedo_0)

      Cijk1albedo = neighbours * np.log((sijk1albedo * 10000) + 1) * np.log((tijk1albedo * 10000) + 1)
      Cijk2albedo = neighbours * np.log((sijk2albedo * 10000) + 1) * np.log((tijk2albedo * 10000) + 1)
        
      divisao1albedo = np.where(Cijk1albedo == 0.0, 0, 1 / Cijk1albedo)
      divisao2albedo = np.where(Cijk2albedo == 0.0, 0, 1 / Cijk2albedo)

      Wijk1albedo = divisao1albedo / (divisao1albedo.sum(0) + divisao2albedo.sum(0))
      Wijk2albedo = divisao2albedo / (divisao1albedo.sum(0) + divisao2albedo.sum(0))      
      
      M_L_M_1albedo = slices_m_albedo_0[sc] + slices_l_albedo_1[sc] - slices_m_albedo_1[sc]
      M_L_M_2albedo = slices_m_albedo_0[sc] + slices_l_albedo_2[sc] - slices_m_albedo_2[sc]
        
      if n + lp + i <= ysize:
        Lalbedo = np.zeros((n, xsize-lp), np.float)
      else:
        linhas = (ysize - lp) - (((ysize - lp)//n)*n)
        Lalbedo = np.zeros((linhas, xsize-lp), np.float)
      for m in range(0, ns):
          Lalbedo = Lalbedo + ((Wijk1albedo[m] * M_L_M_1albedo) + (Wijk2albedo[m] * M_L_M_2albedo))

      Lalbedo = np.where(tijk1albedo[sc] == 0.0, M_L_M_1albedo, Lalbedo)
      Lalbedo = np.where(sijk1albedo[sc] == 0.0, M_L_M_1albedo, Lalbedo)
      Lalbedo = np.where(tijk2albedo[sc] == 0.0, M_L_M_2albedo, Lalbedo)
      Lalbedo = np.where(sijk1albedo[sc] == 0.0, M_L_M_2albedo, Lalbedo)

      out_data_albedo = np.ones(in_data_lv.shape, np.float32) * -99
      out_data_albedo[lsp:-lsp, lsp:-lsp] = Lalbedo

      if i == 0:
        out_band_albedo.WriteArray(out_data_albedo)
      else:
        out_band_albedo.WriteArray(out_data_albedo[lsp:], 0, i + lsp)

#Creating the Slices for Tup
      slices_l_tsup_1 = make_slices(id_l_tsup_1, (mw, mw))
      slices_m_tsup_1 = make_slices(id_m_tsup_1, (mw, mw))
      slices_l_tsup_2 = make_slices(id_l_tsup_2, (mw, mw))
      slices_m_tsup_2 = make_slices(id_m_tsup_2, (mw, mw))
      slices_m_tsup_0 = make_slices(id_m_tsup_0, (mw, mw))

      slices_l_tsup_1 = np.ma.masked_where(neighbours < 0.0, slices_l_tsup_1)
      slices_m_tsup_1 = np.ma.masked_where(neighbours < 0.0, slices_m_tsup_1)
      slices_l_tsup_2 = np.ma.masked_where(neighbours < 0.0, slices_l_tsup_2)
      slices_m_tsup_2 = np.ma.masked_where(neighbours < 0.0, slices_m_tsup_2)
      slices_m_tsup_0 = np.ma.masked_where(neighbours < 0.0, slices_m_tsup_0)

# EXECUTION OF STARFM FOR TSUP
      sijk1tsup = abs(slices_l_tsup_1 - slices_m_tsup_1)
      tijk1tsup = abs(slices_m_tsup_1 - slices_m_tsup_0)
      sijk2tsup = abs(slices_l_tsup_2 - slices_m_tsup_2)
      tijk2tsup = abs(slices_m_tsup_2 - slices_m_tsup_0)

      Cijk1tsup = neighbours * np.log((sijk1tsup * 10000) + 1) * np.log((tijk1tsup * 10000) + 1)
      Cijk2tsup = neighbours * np.log((sijk2tsup * 10000) + 1) * np.log((tijk2tsup * 10000) + 1)
        
      divisao1tsup = np.where(Cijk1tsup == 0.0, 0, 1 / Cijk1tsup)
      divisao2tsup = np.where(Cijk2tsup == 0.0, 0, 1 / Cijk2tsup)

      Wijk1tsup = divisao1tsup / (divisao1tsup.sum(0) + divisao2tsup.sum(0))
      Wijk2tsup = divisao2tsup / (divisao1tsup.sum(0) + divisao2tsup.sum(0))
        
      M_L_M_1tsup = slices_m_tsup_0[sc] + slices_l_tsup_1[sc] - slices_m_tsup_1[sc]
      M_L_M_2tsup = slices_m_tsup_0[sc] + slices_l_tsup_2[sc] - slices_m_tsup_2[sc]
        
      if n + lp + i <= ysize:
        Ltsup = np.zeros((n, xsize-lp), np.float)
      else:
        linhas = (ysize - lp) - (((ysize - lp)//n)*n)
        Ltsup = np.zeros((linhas, xsize-lp), np.float)
      for m in range(0, ns):
          Ltsup = Ltsup + ((Wijk1tsup[m] * M_L_M_1tsup) + (Wijk2tsup[m] * M_L_M_2tsup))

      Ltsup = np.where(tijk1tsup[sc] == 0.0, M_L_M_1tsup, Ltsup)
      Ltsup = np.where(sijk1tsup[sc] == 0.0, M_L_M_1tsup, Ltsup)
      Ltsup = np.where(tijk2tsup[sc] == 0.0, M_L_M_2tsup, Ltsup)
      Ltsup = np.where(sijk1tsup[sc] == 0.0, M_L_M_2tsup, Ltsup)

      out_data_tsup = np.ones(in_data_lv.shape, np.float32) * -99
      out_data_tsup[lsp:-lsp, lsp:-lsp] = Ltsup

      if i == 0:
        out_band_tsup.WriteArray(out_data_tsup)
      else:
        out_band_tsup.WriteArray(out_data_tsup[lsp:], 0, i + lsp)
    
      print('This is the iteration number {}. There are {} interactions left.'.format(int(i/n)+1, int(ysize//n)+1 - int(i/n)+1))

  print(time.time() - start_time, "seconds")
  print('#'*150)
  print('Exporting the images of the date  {}'.format( lista_de_datas_modis_to[dat]))
  print('#'*150)

  out_band_lv.FlushCache()
  out_band_lv.ComputeStatistics(False)##

  out_band_liv.FlushCache()
  out_band_liv.ComputeStatistics(False)

  out_band_albedo.FlushCache()
  out_band_albedo.ComputeStatistics(False)

  out_band_tsup.FlushCache()
  out_band_tsup.ComputeStatistics(False)

  del out_ds_lv, out_ds_liv, out_ds_albedo, out_ds_tsup, in_ds, in_ds_mv, in_ds_k2, in_ds_mv_k2, in_ds_mv_ko

# SEBAL/STARFM

In [ ]:
os.chdir(r"/content/drive/MyDrive/colab_imagens/Pet_Juaz")


area = ee.Geometry.Polygon(
        [[[-40.721, -9.768],
          [-40.174, -9.768],
          [-40.174, -9.015],
          [-40.721, -9.015]]])

lat = -9.457#///////// Insert the average Latitude of the scene!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

###################TERRA###################
'''lista_de_DOA = [167, 200, 203, 232, 233, 237, 240, 241, 251, 253, 255, 272, 278, 290, 294, 301, 310, 317, 318, 319, 321, 335, 338, 342, 345, 349, 12, 60, 66, 100, 117, 139]
lista_de_ta = [30.48, 26.1, 26.99, 26.94, 28.57, 28.09, 30.24, 26.73, 29.54, 28.54, 28.5, 30.77, 32.23, 29.55, 29.54, 30.69, 33.3, 34.29, 24.46, 34.33, 32.09, 31.18, 38.1, 32.54, 31.84, 29.72, 30.03, 30.78, 29.8, 31.74, 30.89, 29.92]
lista_de_ur = [46.27, 50.18, 46.4, 52.15, 52.25, 46.34, 41.18, 48.05, 34.98, 46.54, 50.84, 37.14, 40.13, 40.33, 44.6, 35.34, 36.62, 31.49, 60.8, 35.3, 41.77, 36.68, 22.03, 41.77, 39.94, 38.33, 55.44, 51.14, 54.13, 46.77, 43.15, 52.71]
lista_de_pa = [97.214, 97.554, 97.408, 97.335, 97.306, 97.309, 97.279, 97.564, 97.243, 97.297, 97.316, 97.03, 97, 97.267, 97.156, 97.123, 96.976, 96.855, 96.908, 97.03, 96.803, 97.176, 96.575, 97.061, 97.152, 97.104, 96.964, 97.062, 97.117, 96.972, 97.007, 97.164]
lista_de_u = [3.038, 3.428, 5.11, 2.854, 1.873, 4.505, 4.045, 4.414, 4.675, 5.493, 2.658, 4.471, 4.024, 6.015, 4.704, 2.704, 3.63, 3.215, 1.354, 2.019, 5.442, 4.491, 2.097, 4.069, 3.643, 4.438, 4.61, 5.384, 3.357, 3.516, 3.519, 2.2]

lista_de_rs_24 = [21.6396, 23.04, 23.6736, 25.7796, 25.85, 25.97, 25.99, 25.92, 27.49, 26.65, 27.69, 29.00, 14.10, 30.14, 29.88, 30.45, 29.96, 30.24, 29.76, 29.99, 30.00, 29.00, 27.95, 28.79, 29.90, 29.83, 26.836, 29.066, 26.881, 24.554, 24.392, 22.18]

lista_de_precipitation60 = [101.85, 7.308, 7.308, 6.096, 6.096, 6.096, 6.096, 6.096, 5.334, 5.334, 5.334, 0.762, 0.762, 10.418, 10.418, 10.418, 10.418, 10.418, 10.418, 10.418, 10.672, 11.688, 11.688, 2.286, 2.286, 2.54, 90.674, 30.986, 29.716, 39.622, 13.97, 4.318]
lista_de_ETr60 = [360.28, 324.3, 329.8, 252.03, 254.19, 262.82, 268.77, 270.59, 289.06, 292.25, 295.75, 308.39, 308.3, 296.52, 297.68, 299.84, 291.41, 291.57, 292.82, 294.6, 296.76, 292.83, 297.31, 297.37, 303.35, 312.45, 290.43, 310.12, 316.14, 344.04, 334.33, 307.47]
'''
###################AQUA###################
'''lista_de_DOA = [210, 217, 245, 247, 249, 254, 256, 265, 267, 269, 275, 277, 283, 286, 291, 295, 299, 302, 315, 334, 336, 347, 39, 67]
lista_de_ta = [30.57, 30.65, 33.84, 32.78, 34.88, 32.69, 34.7, 33.18, 33.13, 34.36, 33.59, 34.94, 33.01, 34.05, 34.37, 36.6, 34.12, 35.8, 34.43, 36.39, 35.46, 35.25, 30.84, 33.85]
lista_de_ur = [32.41, 29.29, 21.34, 30.41, 24.43, 26.86, 26.27, 28.9, 25.06, 32.05, 24.47, 31.49, 29.59, 26.14, 21.84, 22.79, 27.39, 28.24, 22.4, 20.79, 26.86, 27.03, 41.45, 37.83]
lista_de_pa = [97.214, 97.184, 96.924, 96.981, 96.933, 96.824, 96.915, 96.915, 96.945, 96.788, 96.764, 96.703, 96.987, 96.957, 96.902, 96.519, 96.903, 96.576, 96.727, 96.815, 96.7, 96.785, 97.2, 96.749]
lista_de_u = [5.012, 3.305, 2.064, 1.76, 3.467, 1.94, 3.341, 3.075, 3.251, 5.66, 1.961, 3.013, 4.339, 5.325, 3.493, 3.664, 3.083, 2.534, 3.31, 6.094, 4.906, 5.431, 3.221, 3.122]

lista_de_rs_24 = [22.9392, 24.66, 27.69, 26.59, 26.72, 27.60, 24.34, 25.76, 27.95, 28.72, 28.92, 28.72, 28.73, 30.50, 31.24, 29.66, 29.94, 29.83, 31.02, 30.67, 30.07, 30.16, 29.389, 28.25]

lista_de_precipitation60 = [0.762, 5.842, 6.096, 6.096, 6.096, 5.334, 5.334, 0.762, 0.762, 0.762, 0.762, 0.762, 10.418, 10.418, 10.418, 10.418, 10.418, 10.418, 10.418, 11.688, 11.688, 2.286, 304.028, 29.716]
lista_de_ETr60 = [245.52, 246.23, 278.21, 282.38, 286.39, 294.21, 297.38, 304.66, 307.39, 307.3, 310.41, 308.19, 309.82, 303.6, 298.02, 298.69, 300.3, 300.13, 292.33, 293.53, 293.82, 309.32, 279.52, 318.41]
'''
lista_de_DOA = [299, 302, 315, 334, 336, 347, 39, 67]
lista_de_ta = [34.12, 35.8, 34.43, 36.39, 35.46, 35.25, 30.84, 33.85]
lista_de_ur = [27.39, 28.24, 22.4, 20.79, 26.86, 27.03, 41.45, 37.83]
lista_de_pa = [96.903, 96.576, 96.727, 96.815, 96.7, 96.785, 97.2, 96.749]
lista_de_u = [ 3.083, 2.534, 3.31, 6.094, 4.906, 5.431, 3.221, 3.122]

lista_de_rs_24 = [29.94, 29.83, 31.02, 30.67, 30.07, 30.16, 29.389, 28.25]

lista_de_precipitation60 = [ 10.418, 10.418, 10.418, 11.688, 11.688, 2.286, 304.028, 29.716]
lista_de_ETr60 = [300.3, 300.13, 292.33, 293.53, 293.82, 309.32, 279.52, 318.41]




###################TERRA###################
#lista_de_datas = ['2015_06_16', '2015_07_19', '2015_07_22', '2015_08_20', '2015_08_21', '2015_08_25', '2015_08_28', '2015_08_29', '2015_09_08', '2015_09_10', '2015_09_12', '2015_09_29', '2015_10_05', '2015_10_17', '2015_10_21', '2015_10_28', '2015_11_06', '2015_11_13', '2015_11_14', '2015_11_15', '2015_11_17', '2015_12_01', '2015_12_04', '2015_12_08', '2015_12_11', '2015_12_15', '2016_01_12', '2016_02_29', '2016_03_06', '2016_04_09', '2016_04_26', '2016_05_18']

###################AQUA###################
#lista_de_datas = ['2015_07_29', '2015_08_05', '2015_09_02', '2015_09_04', '2015_09_06', '2015_09_11', '2015_09_13', '2015_09_22', '2015_09_24', '2015_09_26', '2015_10_02', '2015_10_04', '2015_10_10', '2015_10_13', '2015_10_18', '2015_10_22', '2015_10_26', '2015_10_29', '2015_11_11', '2015_11_30', '2015_12_02', '2015_12_13', '2016_02_08', '2016_03_07']
lista_de_datas = ['2015_10_26', '2015_10_29', '2015_11_11', '2015_11_30', '2015_12_02', '2015_12_13', '2016_02_08', '2016_03_07']

###################TERRA###################
#nome_ga = ee.String('MODIS/006/MOD09GA/')

####################AQUA###################
nome_ga = ee.String('MODIS/006/MYD09GA/')


numero_de_datas = len(lista_de_datas)


for dat in range(0, numero_de_datas):

  data = ee.String(lista_de_datas[dat])
  pegar_ga_1 = nome_ga.cat(data)
  ang_zen = ee.Image(pegar_ga_1.getInfo())# // Insert the date (YEAR-MONTH-DAY)!!!!!!!!!!!!!!!!


  argumento = 2*np.pi*(lista_de_DOA[dat]-1)/365
  dr = 1.000110+0.034221*np.cos(argumento)+0.001280*np.sin(argumento)+0.000719*np.cos(2*argumento)+0.000077*np.sin(2*argumento)
  print ('dr is: ', dr)
 
  cos_z_img = ang_zen.select(['SolarZenith']).multiply(0.01).multiply(np.pi).divide(180).cos()
 
  cos_z_red = cos_z_img.reduceRegion(
    ee.Reducer.mean(),
    area,
    30)
 
  cos_z = cos_z_red.getInfo()['SolarZenith']
 
  elevacao = ((np.pi/2) - math.acos(cos_z))*180/np.pi

  print('Essa é a elevação: ', elevacao, 'Esse é o cos(Z): ', cos_z, 'Esse é o dr: ', dr)


#albedo_col2_ and reflectance_col2_
  albedo_open = gdal.Open('albedo_umpar_' + lista_de_datas[dat] + '.tif')
  albedo_in_band = albedo_open.GetRasterBand(1)
  albedo_in_data = albedo_in_band.ReadAsArray().astype(np.float)

  reflectance_open = gdal.Open('ref_iv_umpar_' + lista_de_datas[dat] + '.tif')
  NIR_in_band = reflectance_open.GetRasterBand(1)
  NIR_in_data = NIR_in_band.ReadAsArray().astype(np.float)

  reflectance_verm_open = gdal.Open('ref_verm_umpar_' + lista_de_datas[dat] + '.tif')
  RED_in_band = reflectance_verm_open.GetRasterBand(1)
  RED_in_data = RED_in_band.ReadAsArray().astype(np.float)

  tsup_open = gdal.Open('tsup_umpar_' + lista_de_datas[dat] + '.tif')
  tsup_in_band = tsup_open.GetRasterBand(1)
  tsup_in_data = tsup_in_band.ReadAsArray().astype(np.float)


  nlcd_open = gdal.Open('mapbiomas_2015_2.tif')
  nlcd_in_band = nlcd_open.GetRasterBand(1)
  nlcd_in_data = nlcd_in_band.ReadAsArray().astype(np.float)


  albedo = np.ma.masked_where(albedo_in_data < 0, albedo_in_data)
  albedo = np.ma.masked_where(albedo > 1, albedo)
 
  NIR = np.where(NIR_in_data < 0.0, 0.0, NIR_in_data)
  NIR = np.ma.masked_where(NIR <= 0, NIR)
 
  RED = np.where(RED_in_data < 0.0, 0.0, RED_in_data)
  RED = np.ma.masked_where(RED <= 0, RED)
 
  tsup = np.ma.masked_where(tsup_in_data < 273, tsup_in_data)
  tsup = np.ma.masked_where(tsup > 340, tsup)
 
 
#EXPORTS#####################################################

  nome_ndvi = '_ndvi_starfm.tif'
  export_ndvi = lista_de_datas[dat] + nome_ndvi
 
  nome_et24 = '_et_24_starfm_umpar_.tif'
  export_et24 = lista_de_datas[dat] + nome_et24

  nome_et24_alf = '_et_24_starfm_alf.tif'
  export_et24_alf = lista_de_datas[dat] + nome_et24_alf

  nome_et24_gr = '_et_24_starfm_gr.tif'
  export_et24_gr = lista_de_datas[dat] + nome_et24_gr

  nome_le = '_le_starfm.tif'
  export_le = lista_de_datas[dat] + nome_le

  nome_g = '_g_starfm.tif'
  export_g = lista_de_datas[dat] + nome_g

  nome_rn = '_rn_starfm.tif'
  export_rn = lista_de_datas[dat] + nome_rn 

  nome_h = '_h_starfm.tif'
  export_h = lista_de_datas[dat] + nome_h

  nome_fe = '_fe_starfm.tif'
  export_fe = lista_de_datas[dat] + nome_fe

  nome_tsup_q = '_tsup_pixls_quentes_col2.tif'
  export_tsup_q = lista_de_datas[dat] + nome_tsup_q

  nome_tsup_f = '_tsup_pixls_frios_col2.tif'
  export_tsup_f = lista_de_datas[dat] + nome_tsup_f

  nome_tsup = '_tsup_starfm.tif'
  export_tsup = lista_de_datas[dat] + nome_tsup

  nome_albedo = '_albedo_starfm.tif'
  export_albedo = lista_de_datas[dat] + nome_albedo

  nome_rah = '_rah_starfm.tif'
  export_rah = lista_de_datas[dat] + nome_rah

  nome_rn_24 = '_rn_24_starfm.tif'
  export_rn_24 = lista_de_datas[dat] + nome_rn_24

#CALCULUS####################################################
  Tfac = 2.6 - (13*(lista_de_precipitation60[dat]/lista_de_ETr60[dat]))
  print(Tfac)

  ndvi = (NIR - RED) / (NIR + RED)
  savi = (1.1 * (NIR - RED)) / (0.1 + NIR + RED)

  iaf = np.where(savi < 0.688000, np.log((0.69 - savi) / 0.59) * (-1) / 0.91, 6)
  iafcorrigido = np.where(savi < 0.000001, 0.000001, iaf)
  iafcorrigido = np.where(iafcorrigido < 0.000001, 0.000001, iaf)

  e_o = np.where(iafcorrigido > 2.999999, 0.98, (0.059 * np.log(ndvi)) + 1.004)
  e_ocorrigido = np.where(iafcorrigido < 0.000001, 0.985, e_o)

  e_nb = np.where(iafcorrigido > 2.999999, 0.98, 0.97 + (0.0033 * iafcorrigido))
  e_nbcorrigido = np.where(iafcorrigido < 0.000001, 0.99, e_nb)

  rol_emi = e_o * 0.0000000567 * tsup**4

  e_a = 0.61078 * (np.exp((17.269 * lista_de_ta[dat])/(237.3 + lista_de_ta[dat]))) * lista_de_ur[dat]/100
  w = (10 * (0.14 * 10 * e_a * (lista_de_pa[dat]/101))) + 0.21
  transmitancia = 0.35 + 0.627 * (np.exp((0.00146*(-1)*lista_de_pa[dat]/cos_z) - 0.075 * ((w/cos_z)**0.4)))

  roc = 1367 * dr * cos_z * transmitancia
  print ('This is the shortwave radiation: ', roc)

  e_atm = 0.625 * (((1000 * e_a) / (273.15 + lista_de_ta[dat])) ** 0.131)

  rol_atm = e_atm * 0.0000000567 * (lista_de_ta[dat] + 273.15)**4
  print ('The Rad_atm is: ', rol_atm)

  rn = (1 - albedo) * roc - rol_emi + e_ocorrigido * rol_atm 

  g_corrigido = np.where(ndvi < 0.05, 0.3 * rn, ((tsup - 273.15) * (0.0038 + 0.0074 * albedo) * (1 - 0.98 * (ndvi ** 4))) * rn)

#//#//#//#//#//#//#//#//#//#// ENERGY BALANCE#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//COLD PIXEL - CIMEC

  p95 = np.percentile(ndvi[ndvi > 0], 95)
  print('This is the 95th percentile of the NDVI: ', p95)

  f1_frio_0 = np.ma.masked_where(nlcd_in_data != 36, ndvi)
  f1_frio = np.ma.masked_where(f1_frio_0 < p95, tsup)

  fff = f1_frio[f1_frio > 274]

  p20 =  np.percentile(fff[fff < 350], 20)
  print('This is the 20th percentile of Tsup: ', p20)

  f2_frio = np.ma.masked_where(f1_frio > p20, f1_frio)

  f2_frio_med = np.nanmean(f2_frio[f2_frio > 0])
  print('This is the average for the cold pixel: ', f2_frio_med)


  f3_frio = np.ma.masked_where(f2_frio > (f2_frio_med + 0.3), f2_frio)
  f3_frio = np.ma.masked_where(f2_frio < (f2_frio_med - 0.3), f3_frio)

  albedo_chave = 0.001343 * elevacao + (0.3281 * np.exp((-0.0188) * elevacao))
  print('The key-albedo is: ', albedo_chave)

  f4_frio = np.ma.masked_where(albedo > (albedo_chave + 0.02), f3_frio)
  f4_frio = np.ma.masked_where(albedo < (albedo_chave - 0.02), f4_frio)

  p_frio_ok = np.min(f4_frio[f4_frio > 0])

  print('This is the CIMEC cold pixel: ', p_frio_ok)

#//HOT PIXEL - CIMEC
  p5 = np.percentile(ndvi[ndvi>0], 5)
  print('This is the 5th percentile of the NDVI: ', p5)
  f1_quente = np.ma.masked_where(ndvi > p5, tsup)

  f1_quente = np.ma.masked_where(f1_quente > 400, f1_quente)

  qqq = f1_quente[f1_quente > 274]

  p80 = np.percentile(qqq[qqq < 350], 80)
  print('This is the 80th percentile of the Tsup: ', p80)
  f2_quente = np.ma.masked_where(f1_quente < p80, f1_quente)

  f2_quente_med = np.mean(f2_quente)
  f2_quente_med_nan = np.nanmean(f2_quente)
  print('This is the average Ts of the pixels: ', f2_quente_med)
  print('This is the average Ts of the pixels: ', f2_quente_med_nan)
  f2_quente_med_2 = f2_quente_med_nan - Tfac
  print('This is the average of the Ts of the pixels - Tfac: ', f2_quente_med_2)
  
  f3_quente = np.ma.masked_where(f2_quente > (f2_quente_med_2 + 2.0), f2_quente)
  f3_quente = np.ma.masked_where(f3_quente < (f2_quente_med_2 - 2.0), f3_quente)

  p_quente_ok = np.max(f3_quente[f3_quente > 0])

  print('This is CIMEC hot pixel: ', p_quente_ok)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///

#//SAVI OF THE HOT PIXEL
  savi_pq = np.ma.masked_where(ndvi > p5, tsup)
  savi_pq = np.ma.masked_where(savi_pq != p_quente_ok, savi)
  savi_quente = savi_pq.max()
  print ('The SAVI of the hot pixel is: ', savi_quente)

#//Rn OF THE HOT PIXEL
  rn_pq = np.ma.masked_where(ndvi > p5, tsup)
  rn_pq = np.ma.masked_where(rn_pq != p_quente_ok, rn)
  rn_quente = rn_pq.max()
  print ('The Rn of the hot pixel is: ', rn_quente)

#//G OF THE HOT PIXEL
  g_pq = np.ma.masked_where(ndvi > p5, tsup)
  g_pq = np.ma.masked_where(g_pq != p_quente_ok, g_corrigido)
  g_quente = g_pq.max()
  print ('The G of the hot pixel is: ', g_quente)

#//NDVI OF THE HOT PIXEL
  ndvi_pq = np.ma.masked_where(ndvi > p5, tsup)
  ndvi_pq = np.ma.masked_where(ndvi_pq != p_quente_ok, ndvi)
  ndvi_quente = ndvi_pq.max()

#//Albedo OF THE HOT PIXEL
  albedo_pq = np.ma.masked_where(ndvi > p5, tsup)
  albedo_pq = np.ma.masked_where(albedo_pq != p_quente_ok, albedo)
  albedo_quente = albedo_pq.max()

#//calculating linear regression coefficients dT = a + bT
  u_estrela1 = lista_de_u[dat] * 0.41/np.log(10/0.024)# Attention in this value 10 here in the logarithm! 10 stands for wind measurement height (10 m)
  u200 = u_estrela1 * np.log(200/0.024)/0.41
  zom = np.exp(-2.21+(0.26*ndvi_quente/albedo_quente))
  u_estrela = (u200*0.41)/np.log(200/zom)
  rah = (np.log(2/0.1))/(0.41 * u_estrela)
  b = (rn_quente - g_quente)*rah / (1.15*1004*(p_quente_ok - p_frio_ok))

  a = (-1)*b*(p_frio_ok - 273.15)
  dT = a + b *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a + (b * (p_quente_ok - 273.15)))/rah

  print ('The sensible heat flux H is: ', H)
  rn_g = rn_quente - g_quente
  print ('Rn - G is : ', rn_g)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculating in the images#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  zom_img = np.exp(-2.21+(0.26*ndvi/albedo))
  u_estrela_img = (u200*0.41)/np.log(200/zom_img)
  rah_img = (np.log(2/0.1))/(0.41 * u_estrela_img)
  H_img = 1.15 * 1004 * (a + (b * (tsup - 273.15)))/rah_img

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//START OF THE ITERATIVE PROCESS.#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//1st CORRECTION#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#///
  L = (-1)*1.15*1004*(u_estrela**3)*p_quente_ok/(0.41*9.81*H)
  if L <= 0.0:
    x_0_1m = ((1-16)*0.1/L)**0.25
    x_2m = ((1-16)*2/L)**0.25
    x_200m = ((1-16)*200/L)**0.25

    psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)
    psi_h2 = 2*np.log((1+(x_2m**2))/2)
    psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi
  else:
    psi_h0_1 = (-5) * (0.1/L)
    psi_h2 = (-5) * (2/L)
    psi_m200 = (-5) * (200/L)

  u_estrela_corr_1 = u200*0.41/((np.log(200/zom))-psi_m200)
  rah_corr_1 = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*u_estrela_corr_1)

  b_corr = (rn_quente - g_quente)*rah_corr_1 / (1.15*1004*(p_quente_ok - p_frio_ok))
  a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
  print('a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)
  dT_corr = a_corr + b_corr *(p_quente_ok - 273.15)
  H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/rah_corr_1

  print ('O Fluxo de calor sensível H = : ', H)

#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//#//calculating 1st CORRECTION on images #//#//#//#//#//#//#//#//#//#//#//#//#//#//#//
  L_img = (-1)*1.15*1004*(u_estrela_img**3)*tsup/(0.41*9.81*H_img)
  
  x_0_1m_img = ((1-16)*0.1/L_img)**0.25
  x_2m_img = ((1-16)*2/L_img)**0.25
  x_200m_img = ((1-16)*200/L_img)**0.25

  psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
  psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
  psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                          (-1)*5*(200/L_img))
  u_estrela_corr_img_1 = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
  rah_corr_img_1 = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * u_estrela_corr_img_1)
  H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/rah_corr_img_1

  err_rel = abs(rah_corr_1 - rah)*100/rah
  print('a vale: ', a_corr, 'b vale: ', b_corr, 'O Erro relativo é: ', err_rel)
  print('H é: ', H)

  lista_u_star = []
  lista_rah = []
  lista_u_star_img = []
  lista_rah_img = []

  lista_u_star.append(u_estrela_corr_1)
  
  lista_rah.append(rah_corr_1)
    lista_u_star_img.append(u_estrela_corr_img_1)
  lista_rah_img.append(rah_corr_img_1)


  rodadas = 1

  while err_rel > 1.5:
    rodadas += 1
    lista_u_star.append('u_estrela_corr_{}'.format(rodadas))
    
    lista_rah.append('rah_corr_{}'.format(rodadas))
    
    lista_u_star_img.append('u_estrela_corr_img_{}'.format(rodadas))    
    lista_rah_img.append('rah_corr_img_{}'.format(rodadas))
 
    L = (-1)*1.15*1004*((lista_u_star[0])**3)*p_quente_ok/(0.41*9.81*H)
 
    if L <= 0.0:
      x_0_1m = ((1-16)*0.1/L)**0.25      
      x_2m = ((1-16)*2/L)**0.25      
      x_200m = ((1-16)*200/L)**0.25      
 
      psi_h0_1 = 2*np.log((1+(x_0_1m**2))/2)      
      psi_h2 = 2*np.log((1+(x_2m**2))/2)      
      psi_m200 = 2*np.log((1+x_200m)/2)+np.log((1+x_200m**2)/2)-2*np.arctan(x_200m)+0.5*np.pi      
    else:
      psi_h0_1 = (-5) * (0.1/L)
      psi_h2 = (-5) * (2/L)
      psi_m200 = (-5) * (200/L)
    
    lista_u_star[1] = u200*0.41/((np.log(200/zom))-psi_m200)
    lista_rah[1] = (np.log(2/0.1)-psi_h2+psi_h0_1)/(0.41*(lista_u_star[1]))

    b_corr = (rn_quente - g_quente)*lista_rah[1] / (1.15*1004*(p_quente_ok - p_frio_ok))
    
    a_corr = (-1)*b_corr*(p_frio_ok - 273.15)
    H = 1.15 * 1004 * (a_corr + (b_corr * (p_quente_ok - 273.15)))/lista_rah[1]

    #calculating in the images:
    L_img = (-1)*1.15*1004*(((lista_u_star_img[0]))**3)*tsup/(0.41*9.81*H_corr_img)
    x_0_1m_img = ((1-16)*0.1/L_img)**0.25
    x_2m_img = ((1-16)*2/L_img)**0.25
    x_200m_img = ((1-16)*200/L_img)**0.25
 
    psi_h0_1_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_0_1m_img**2))/2), (-1)*5*(0.1/L_img))
    psi_h2_img_c = np.where(L_img < 0.0, 2*np.log((1+(x_2m_img**2))/2), (-1)*5*(2/L_img))
    psi_m200_img_c = np.where(L_img < 0.0, 2*np.log((1+x_200m_img)/2)+np.log((1+(x_200m_img**2))/2)-(2*np.arctan(x_200m_img))+0.5*(np.pi),
                              (-1)*5*(200/L_img))
    lista_u_star_img[1] = (u200*0.41)/((np.log(200/zom_img))-psi_m200_img_c)
    lista_rah_img[1] = (np.log(2/0.1)-psi_h2_img_c+psi_h0_1_img_c)/(0.41 * lista_u_star_img[1])
    H_corr_img = 1.15 * 1004 * (a_corr + (b_corr * (tsup - 273.15)))/lista_rah_img[1]
       
    
    err_rel = abs(lista_rah[1] - lista_rah[0])*100/lista_rah[1]
    
    print('Round',rodadas,' a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)
    print ('H is: ', H)

    #deleting the first item from each list
    lista_u_star.pop(0)
    lista_rah.pop(0)
    lista_u_star_img.pop(0)
    lista_rah_img.pop(0)

  print('End of the process, a is equivalent to: ', a_corr, 'b is equivalent to: ', b_corr, 'The relative error is: ', err_rel)


#//Calculating Latent Heat Flux (LE) and ET_24h

  LE = rn - g_corrigido - H_corr_img
  LE = np.where(LE < 0.0, 0.0, LE)
  fe = LE / (rn - g_corrigido)

  del_ = 0.006918-0.399912*np.cos(2*np.pi*(lista_de_DOA[dat]/365))+0.070257*np.sin(2*np.pi*(lista_de_DOA[dat]/365))-0.006758*np.cos(2*2*np.pi*(lista_de_DOA[dat]/365))+0.000907*np.sin(2*2*np.pi*(lista_de_DOA[dat]/365))-0.002697*np.cos(3*2*np.pi*(lista_de_DOA[dat]/365))+0.00148*np.sin(3*2*np.pi*(lista_de_DOA[dat]/365))
  fi = lat * np.pi/180
  H_hor = np.arccos(-np.tan(fi)*np.tan(del_))
  rs_24_toa = 118.08*dr*(H_hor*np.sin(fi)*np.sin(del_)+np.cos(fi)*np.cos(del_)*np.sin(H_hor))/np.pi
  print('Rs_TOA = : ', rs_24_toa)

  tsw_24 = lista_de_rs_24[dat] / rs_24_toa
  print('tsw_24 = : ', tsw_24)
  rn_24 = (1-albedo)*lista_de_rs_24[dat]*11.57407 - 110*tsw_24

  ET_24 = 0.0353*fe*rn_24
  ET_24 = np.ma.masked_where(ET_24 < 0, ET_24)
  ET_24 = ET_24.filled(-99)

  print('#'*150)
  print('The image(s) of the date {} is(are) beeing exported'.format(lista_de_datas[dat]))
  print('#'*150)

  out_ds = make_raster(albedo_open, export_et24, ET_24, gdal.GDT_Float32, -99)

del out_ds